In [2]:
# -*- coding: utf-8 -*-
import json
import random
import re
import heapq
from math import ceil, floor, exp
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Callable, Any
import argparse
import sys

# ======================== CẤU HÌNH CHUNG ========================
SEED = 42
random.seed(SEED)

SCALE = 1000            # scale cho khối lượng
CHUNK_SAFETY = 0.95     # cắt chunk <= 95% min cap một trip
HUGE = 1_000_000_000

# ALNS
SEGMENT_LENGTH = 50
RHO = 0.6
REWARD_GLOBAL_BEST = 6.0
REWARD_IMPROVED = 3.0
REWARD_ACCEPTED = 1.0
EARLY_STOP_PATIENCE = 2000

# ======================== TIỆN ÍCH DATA ========================
def load_json(path: Path) -> Any:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def save_json(path: Path, obj: Any) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

def build_id_to_idx(nodes_json) -> Dict[int, int]:
    return {int(n["id"]): idx for idx, n in enumerate(nodes_json["nodes"])}

def get_vendor_demand(nodes_json, GOODS: List[str]) -> Dict[int, Dict[str, float]]:
    out = {}
    for n in nodes_json["nodes"]:
        if n.get("type") != "vendor":
            continue
        vid = int(n["id"])
        gdict = (
            n.get("collected_goods_dict")
            or n.get("collected_goods")
            or n.get("order_dict")
            or n.get("goods_dict")
            or {}
        )
        out[vid] = {g: float(gdict.get(g, 0.0)) for g in GOODS}
    return out

# ======================== XE & CHI PHÍ ========================
def vehicles_of_depot(vehicles_json, depot_internal: int) -> List[dict]:
    return [v for v in vehicles_json["vehicles"] if int(v["depot_id"]) == depot_internal]

def vehicle_capacity_vec(v: dict, GOODS: List[str]) -> List[int]:
    return [int(round(float(v["capacity_dict"][g]) * SCALE)) for g in GOODS]

def vehicle_cost_per_km(v: dict, vehicle_costs_table: Dict[str, float]) -> float:
    t = v.get("vehicle_type", "type_1")
    return float(vehicle_costs_table.get(t, 1.0))

def _parse_vehicle_idx_as_int(v: dict) -> int:
    raw = str(v.get("vehicle_idx", v.get("vehicle_id", "0")))
    m = re.search(r'\d+', raw)
    return int(m.group()) if m else 0

def pick_cheapest_vehicle(vehicles_json, depot_internal: int, GOODS: List[str]):
    vehs = vehicles_of_depot(vehicles_json, depot_internal)
    if not vehs:
        return None, None, None, None
    costs = vehicles_json["meta"]["vehicle_costs"]
    def _cost(v): return vehicle_cost_per_km(v, costs)
    best = min(vehs, key=_cost)
    cap_vec = vehicle_capacity_vec(best, GOODS)
    veh_cost = _cost(best)
    veh_idx = _parse_vehicle_idx_as_int(best)
    return best, cap_vec, veh_cost, veh_idx

# ======================== TOÁN VECTORS ========================
def sub_vector_inplace(a: List[int], b: List[int]) -> None:
    for i in range(len(a)): a[i] -= b[i]

def add_vector_inplace(a: List[int], b: List[int]) -> None:
    for i in range(len(a)): a[i] += b[i]

def leq_vec(a: List[int], b: List[int]) -> bool:
    return all(a[i] <= b[i] for i in range(len(a)))

def fit_alloc_into_cap(alloc_vec: List[int], cap_vec: List[int]) -> Tuple[List[int], List[int]]:
    fit = [min(alloc_vec[i], cap_vec[i]) for i in range(len(alloc_vec))]
    rem = [alloc_vec[i] - fit[i] for i in range(len(alloc_vec))]
    return fit, rem

# ======================== DỮ LIỆU GRAPH & RÚT GỌN CẠNH CẤM ========================
def _norm_val(x) -> float:
    if x is None: return float("inf")
    if isinstance(x, str) and x.lower() == "infinity": return float("inf")
    return float(x)

def build_graph_from_matrix(distance_matrix: List[List[Any]]) -> List[List[float]]:
    n = len(distance_matrix)
    G = [[float("inf")] * n for _ in range(n)]
    for i in range(n):
        G[i][i] = 0.0
        row = distance_matrix[i]
        for j in range(n):
            val = _norm_val(row[j])
            if val < float("inf"):
                G[i][j] = val
    return G

def dijkstra_path(G: List[List[float]], src: int, dst: int) -> Tuple[float, List[int]]:
    n = len(G)
    dist = [float("inf")] * n
    parent = [-1] * n
    dist[src] = 0.0
    pq = [(0.0, src)]
    while pq:
        d, u = heapq.heappop(pq)
        if d != dist[u]: 
            continue
        if u == dst:
            break
        for v in range(n):
            w = G[u][v]
            if w == float("inf"): 
                continue
            nd = d + w
            if nd < dist[v]:
                dist[v] = nd
                parent[v] = u
                heapq.heappush(pq, (nd, v))
    if dist[dst] == float("inf"):
        return float("inf"), []
    # reconstruct path (indices)
    path = []
    cur = dst
    while cur != -1:
        path.append(cur)
        cur = parent[cur]
    path.reverse()
    return dist[dst], path

def preprocess_only_for_blocked_pairs(
    nodes_json, clusters_json, matrix_json, output_path: Path
) -> Tuple[List[List[float]], Dict[Tuple[int,int], List[int]]]:
    """
    - Nhận matrix gốc (chứa 'Infinity' / None cho cạnh cấm).
    - Chỉ rút gọn (tìm đường đi vòng) cho các cặp 'depot<->vendor' bị cấm trực tiếp.
    - Trả về:
        + dist_apsp: ma trận distance mới (chỉ cập nhật ở các cặp đã chữa),
        + path_map: map (node_id_u, node_id_v) -> [id_u, ..., id_v] là đường đi đầy đủ.
    - Đồng thời xuất file JSON '..._floyd.json' như yêu cầu.
    """
    distance_matrix = matrix_json["distance_matrix"]
    id_to_idx = build_id_to_idx(nodes_json)
    idx_to_id = {idx: nid for nid, idx in id_to_idx.items()}
    G = build_graph_from_matrix(distance_matrix)  # adjacency-weighted directed graph

    # Lấy danh sách depot & vendor
    depots = [int(n["id"]) for n in nodes_json["nodes"] if n.get("type") == "depot"]
    vendors = [int(n["id"]) for n in nodes_json["nodes"] if n.get("type") == "vendor"]

    # Chuẩn bị dist & path_map
    n = len(distance_matrix)
    dist_out = [[_norm_val(distance_matrix[i][j]) for j in range(n)] for i in range(n)]
    path_map: Dict[Tuple[int,int], List[int]] = {}

    # Chỉ xét cặp depot<->vendor thuộc cluster assignment (filter nhẹ nhàng)
    assigned = set()
    for c in clusters_json["clusters"]:
        for v in c["assigned_vendors"]:
            assigned.add(int(v))
    
    vendors = [v for v in vendors if v in assigned]

    # Chữa chiều đi & về khi bị chặn trực tiếp (Infinity)
    for depot_id in depots:
        for vendor_id in vendors:
            iu = id_to_idx[depot_id]; iv = id_to_idx[vendor_id]
            # depot -> vendor
            if dist_out[iu][iv] == float("inf"):
                d, p_idx = dijkstra_path(G, iu, iv)
                if d < float("inf"):
                    dist_out[iu][iv] = d
                    # lưu full path theo ID
                    path_map[(depot_id, vendor_id)] = [idx_to_id[k] for k in p_idx]
            # vendor -> depot
            if dist_out[iv][iu] == float("inf"):
                d, p_idx = dijkstra_path(G, iv, iu)
                if d < float("inf"):
                    dist_out[iv][iu] = d
                    path_map[(vendor_id, depot_id)] = [idx_to_id[k] for k in p_idx]

    # Xuất file JSON mới
    dist_out_json = {
        "distance_matrix": [
            [("Infinity" if x == float("inf") else x) for x in row]
            for row in dist_out
        ],
        "path_reconstruction": {
            f"{u}_{v}": path_map[(u,v)] for (u,v) in path_map
        }
    }
    save_json(output_path, dist_out_json)
    return dist_out, path_map

# ======================== DIST ACCESSOR (DÙNG CẢ PATH_MAP) ========================
def dist_accessor(distance_matrix_num, id_to_idx, path_map: Dict[Tuple[int,int], List[int]]) -> Callable[[int,int], float]:
    """
    Nếu (u,v) có path_map -> trả tổng khoảng cách theo path (đảm bảo finite).
    Nếu không, đọc trực tiếp từ matrix (có thể Infinity).
    """
    n = len(distance_matrix_num)

    # tiền tính chiều dài đường đi từ path_map để nhanh hơn
    cache_cost: Dict[Tuple[int,int], float] = {}
    def pair_cost(u: int, v: int) -> float:
        key = (u, v)
        if key in cache_cost:
            return cache_cost[key]
        if key in path_map:
            path_ids = path_map[key]
            # tính tổng theo matrix_num (đã là trọng số cạnh trực tiếp)
            total = 0.0
            for i in range(len(path_ids)-1):
                a = id_to_idx.get(int(path_ids[i]), None)
                b = id_to_idx.get(int(path_ids[i+1]), None)
                if a is None or b is None: 
                    cache_cost[key] = float("inf"); return cache_cost[key]
                w = distance_matrix_num[a][b]
                if w == float("inf"):
                    cache_cost[key] = float("inf"); return cache_cost[key]
                total += w
            cache_cost[key] = total
            return total
        
        # nếu không có path_map, lấy ô trực tiếp
        ia = id_to_idx.get(int(u), None)
        ib = id_to_idx.get(int(v), None)
        if ia is None or ib is None: 
            return float("inf")
        val = distance_matrix_num[ia][ib]
        return val

    def d(a: int, b: int) -> float:
        val = pair_cost(a, b)
        if val == float("inf"):
            return HUGE
        return float(val)

    return d

# ======================== CHUNKING ========================
def integer_split_scaled_safe(
    vendors: List[int],
    demand_map: Dict[int, Dict[str, float]],
    min_cap_scaled: int,
    GOODS: List[str]
):
    if min_cap_scaled <= 0:
        min_cap_scaled = 1
    limit = int(max(1, CHUNK_SAFETY * min_cap_scaled))
    chunks_nodes: List[int] = []
    chunks_qty_scaled: List[int] = []
    for vid in vendors:
        total = float(sum(demand_map.get(vid, {}).get(g, 0.0) for g in GOODS))
        total_scaled = int(round(total * SCALE))
        if total_scaled <= 0:
            continue
        need = ceil(total_scaled / limit)
        base = total_scaled // need
        rem  = total_scaled % need
        for i in range(need):
            take = base + (1 if i < rem else 0)
            take = min(take, limit)
            take = max(1, take)
            chunks_nodes.append(vid)
            chunks_qty_scaled.append(int(take))
    return chunks_nodes, chunks_qty_scaled, None

def allocate_chunk_vector(vendor_remaining_vec: List[int], qty_scaled: int, GOODS: List[str]) -> List[int]:
    total_rem = sum(max(0, x) for x in vendor_remaining_vec)
    if total_rem <= 0:
        return [0]*len(GOODS)
    alloc = []
    for rem_i in vendor_remaining_vec:
        portion = (rem_i / total_rem) if total_rem > 0 else 0.0
        part = int(round(qty_scaled * portion))
        alloc.append(part)
    diff = qty_scaled - sum(alloc)
    i = 0
    while diff != 0 and i < len(alloc):
        if diff > 0:
            alloc[i] += 1; diff -= 1
        else:
            take = min(1, alloc[i])
            alloc[i] -= take; diff += take
        i = (i+1) % len(alloc)
    for i in range(len(alloc)):
        if alloc[i] > vendor_remaining_vec[i]:
            alloc[i] = vendor_remaining_vec[i]
    return alloc

# ======================== CẤU TRÚC LỜI GẢI ========================
@dataclass
class Route:
    depot: int
    vehicle_idx: int
    veh_cost: float
    cap_vec: List[int]
    n_goods: int
    vehicle_type: str = "unknown"               # <-- ADDED
    path: List[int] = field(default_factory=list) # [depot, ..., depot] (chỉ vendor nodes)
    load_vec: List[int] = field(default_factory=list)
    assigned_chunks: List[Dict[str, Any]] = field(default_factory=list)
    dist_cache: float = 0.0

    @staticmethod
    def new(depot_node_id: int, vehicle_idx: int, veh_cost_per_km: float, cap_vec: List[int], GOODS: List[str], vehicle_type: str = "unknown") -> "Route": # <-- MODIFIED
        n_goods = len(GOODS)
        return Route(
            depot=int(depot_node_id),
            vehicle_idx=vehicle_idx,
            veh_cost=float(veh_cost_per_km),
            cap_vec=cap_vec[:],
            n_goods=n_goods,
            vehicle_type=vehicle_type,          # <-- ADDED
            path=[int(depot_node_id), int(depot_node_id)],
            load_vec=[0]*n_goods,
            assigned_chunks=[],
            dist_cache=0.0
        )

    def clone_shallow(self) -> "Route":
        return Route(
            depot=self.depot,
            vehicle_idx=self.vehicle_idx,
            veh_cost=self.veh_cost,
            cap_vec=self.cap_vec[:],
            n_goods=self.n_goods,
            vehicle_type=self.vehicle_type,    # <-- ADDED
            path=self.path[:],
            load_vec=self.load_vec[:],
            assigned_chunks=[{"vendor_id": c["vendor_id"], "alloc": c["alloc"][:]} for c in self.assigned_chunks],
            dist_cache=self.dist_cache
        )

@dataclass
class Solution:
    routes: List[Route]
    unassigned: set              # chỉ chứa "unreachable thật"
    vendor_remaining: Dict[int, List[int]]
    n_goods: int
    cost: float = 0.0

    def clone(self) -> "Solution":
        return Solution(
            routes=[r.clone_shallow() for r in self.routes],
            unassigned=set(self.unassigned),
            vendor_remaining={vid: vec[:] for vid, vec in self.vendor_remaining.items()},
            n_goods=self.n_goods,
            cost=self.cost
        )

# ======================== HÀM CHI PHÍ ========================
def compute_route_distance(route: Route, dist_fn: Callable[[int,int], float]) -> float:
    total = 0.0
    p = route.path
    for i in range(len(p)-1):
        d = dist_fn(p[i], p[i+1])
        if d >= HUGE:
            return float(HUGE)
        total += d
    return total

def recompute_all_cost(sol: Solution, dist_fn: Callable[[int,int], float]) -> float:
    total = 0.0
    for r in sol.routes:
        d = compute_route_distance(r, dist_fn)
        if d >= HUGE:
            sol.cost = float(HUGE)
            return float(HUGE)
        r.dist_cache = d
        total += d * r.veh_cost
    sol.cost = total
    return total

# ======================== TOÁN INSERT ========================
def delta_insert_edge(route: Route, vendor_id: int, dist_fn: Callable[[int,int], float], insert_pos: int) -> float:
    p = route.path
    u = p[insert_pos-1]
    v = p[insert_pos]
    duv = dist_fn(u, v)
    dux = dist_fn(u, vendor_id)
    dxv = dist_fn(vendor_id, v)
    if dux >= HUGE or dxv >= HUGE or duv >= HUGE:
        return float(HUGE)
    return (dux + dxv - duv)

def apply_insert_at(route: Route, vendor_id: int, alloc_vec: List[int], delta_d: float, insert_pos: int) -> None:
    route.path.insert(insert_pos, vendor_id)
    add_vector_inplace(route.load_vec, alloc_vec)
    route.assigned_chunks.append({"vendor_id": vendor_id, "alloc": alloc_vec[:]})
    route.dist_cache += float(delta_d)

def feasible_add_chunk_to_route(route: Route, alloc_vec: List[int]) -> bool:
    new_load = [route.load_vec[i] + alloc_vec[i] for i in range(route.n_goods)]
    return leq_vec(new_load, route.cap_vec)

# ======================== LẤY DANH SÁCH VISITS ========================
def list_all_visits(sol: Solution) -> List[Tuple[int,int,int]]:
    locs: List[Tuple[int,int,int]] = []
    for ri, r in enumerate(sol.routes):
        for pos in range(1, len(r.path)-1):
            vid = r.path[pos]
            if vid != r.depot:
                locs.append((ri, pos, vid))
    return locs

def remove_visit(sol: Solution, ri: int, pos: int) -> Tuple[int, List[int]]:
    r = sol.routes[ri]
    vid = r.path[pos]
    alloc = None
    for i in reversed(range(len(r.assigned_chunks))):
        if r.assigned_chunks[i]["vendor_id"] == vid:
            alloc = r.assigned_chunks.pop(i)["alloc"]
            break
    if alloc is None:
        alloc = [0]*sol.n_goods

    for j in range(sol.n_goods):
        r.load_vec[j] -= alloc[j]
        if vid in sol.vendor_remaining:
            sol.vendor_remaining[vid][j] += alloc[j]

    r.path.pop(pos)
    sol.unassigned.discard(vid)
    return (vid, alloc)

# ======================== DESTROY OPERATORS ========================
def random_removal(sol: Solution, dist_fn, remove_ratio=0.15):
    locs = list_all_visits(sol)
    if not locs:
        return []
    k = max(1, int(len(locs)*remove_ratio))
    picked = random.sample(locs, k)
    picked.sort(key=lambda t: (t[0], t[1]), reverse=True)
    removed = []
    for (ri, pos, vid) in picked:
        removed.append(remove_visit(sol, ri, pos))
    return removed

def worst_removal(sol: Solution, dist_fn, remove_ratio=0.15):
    locs = list_all_visits(sol)
    if not locs:
        return []
    contribs = []
    for (ri, pos, vid) in locs:
        r = sol.routes[ri]
        u = r.path[pos-1]; x = r.path[pos]; v = r.path[pos+1]
        duv = dist_fn(u, v)
        c = (dist_fn(u, x) + dist_fn(x, v) - duv) * r.veh_cost
        contribs.append(((ri, pos, vid), c))
    contribs.sort(key=lambda t: t[1], reverse=True)
    k = max(1, int(len(locs)*remove_ratio))
    picked_idx = [it[0] for it in contribs[:k]]
    picked_idx.sort(key=lambda t: (t[0], t[1]), reverse=True)
    removed = []
    for (ri, pos, vid) in picked_idx:
        removed.append(remove_visit(sol, ri, pos))
    return removed

def shaw_removal(sol: Solution, dist_fn, remove_ratio=0.15):
    locs = list_all_visits(sol)
    if not locs:
        return []
    k = max(1, int(len(locs)*remove_ratio))
    seed_ri, seed_pos, seed_vid = random.choice(locs)
    cand = []
    for (ri, pos, vid) in locs:
        if (ri, pos, vid) == (seed_ri, seed_pos, seed_vid):
            continue
        dist = dist_fn(seed_vid, vid)
        cand.append(((ri, pos, vid), dist))
    cand.sort(key=lambda t: t[1])
    picked_idx = [(seed_ri, seed_pos, seed_vid)] + [c[0] for c in cand[:k-1]]
    picked_idx.sort(key=lambda t: (t[0], t[1]), reverse=True)
    removed = []
    for (ri, pos, vid) in picked_idx:
        removed.append(remove_visit(sol, ri, pos))
    return removed

# ======================== HỖ TRỢ MỞ TRIP MỚI ========================
def _open_new_trip_and_fit(sol: Solution, vid: int, alloc_vec: List[int],
                           vehicles_json, DEPOT_NODE_TO_INTERNAL, dist_fn, GOODS: List[str]) -> Tuple[bool, List[int]]:
    if not sol.routes:
        return False, alloc_vec
    depot_node = sol.routes[0].depot
    depot_internal = DEPOT_NODE_TO_INTERNAL[depot_node]

    best_v, base_cap_vec, veh_cost, veh_idx = pick_cheapest_vehicle(vehicles_json, depot_internal, GOODS)
    if best_v is None:
        return False, alloc_vec
    
    veh_type = best_v.get("vehicle_type", "type_1") # <-- ADDED

    fit_vec, rem_vec = fit_alloc_into_cap(alloc_vec, base_cap_vec)
    if sum(fit_vec) <= 0:
        return False, alloc_vec

    new_r = Route.new(depot_node, veh_idx, veh_cost, base_cap_vec, GOODS, vehicle_type=veh_type) # <-- MODIFIED

    if dist_fn(depot_node, vid) >= HUGE or dist_fn(vid, depot_node) >= HUGE:
        return False, alloc_vec

    delta_d = delta_insert_edge(new_r, vid, dist_fn, insert_pos=1)
    if delta_d >= HUGE:
        return False, alloc_vec

    apply_insert_at(new_r, vid, fit_vec, delta_d, insert_pos=1)
    sol.routes.append(new_r)

    if vid in sol.vendor_remaining:
        for j in range(sol.n_goods):
            sol.vendor_remaining[vid][j] -= fit_vec[j]

    return (sum(rem_vec) == 0), rem_vec

# ======================== REPAIR OPERATORS (FEASIBLE-BẰNG-MỌI-GIÁ) ========================
def greedy_best_position_insertion(sol: Solution, removed_list, dist_fn, GOODS: List[str],
                                   vehicles_json=None, DEPOT_NODE_TO_INTERNAL=None) -> bool:
    processing_list = removed_list[:]
    while processing_list:
        (vid, alloc_vec) = processing_list.pop(0)
        if sum(alloc_vec) <= 0:
            continue
        best = None
        for r in sol.routes:
            if not feasible_add_chunk_to_route(r, alloc_vec):
                continue
            for pos in range(1, len(r.path)):
                delta_d = delta_insert_edge(r, vid, dist_fn, pos)
                if delta_d >= HUGE:
                    continue
                delta_cost = delta_d * r.veh_cost
                if (best is None) or (delta_cost < best[0]):
                    best = (delta_cost, r, delta_d, pos)

        if best is not None:
            _, r, delta_d, pos = best
            apply_insert_at(r, vid, alloc_vec, delta_d, pos)
            if vid in sol.vendor_remaining:
                for j in range(sol.n_goods):
                    sol.vendor_remaining[vid][j] -= alloc_vec[j]
        else:
            done, rem_vec = _open_new_trip_and_fit(sol, vid, alloc_vec, vehicles_json, DEPOT_NODE_TO_INTERNAL, dist_fn, GOODS)
            if not done and sum(rem_vec) > 0:
                processing_list.insert(0, (vid, rem_vec))
    return True

def regret2_best_position_insertion(sol: Solution, removed_list, dist_fn, GOODS: List[str],
                                    vehicles_json=None, DEPOT_NODE_TO_INTERNAL=None) -> bool:
    processing_list = removed_list[:]
    while processing_list:
        candidates = []
        remainders_for_next_round = []

        for (vid, alloc_vec) in processing_list:
            if sum(alloc_vec) <= 0:
                continue
            two_best = []
            for r in sol.routes:
                if not feasible_add_chunk_to_route(r, alloc_vec):
                    continue
                for pos in range(1, len(r.path)):
                    delta_d = delta_insert_edge(r, vid, dist_fn, pos)
                    if delta_d >= HUGE:
                        continue
                    delta_cost = delta_d * r.veh_cost
                    two_best.append((delta_cost, r, delta_d, pos))
            two_best.sort(key=lambda x: x[0])

            if len(two_best) == 0:
                if not sol.routes:
                    remainders_for_next_round.append((vid, alloc_vec))
                    continue
                done, rem_vec = _open_new_trip_and_fit(sol, vid, alloc_vec, vehicles_json, DEPOT_NODE_TO_INTERNAL, dist_fn, GOODS)
                if not done and sum(rem_vec) > 0:
                    remainders_for_next_round.append((vid, rem_vec))
            else:
                candidates.append((vid, alloc_vec, two_best[:2]))

        def regret_score(lst):
            if not lst: return -HUGE
            if len(lst) == 1: return 1e9
            return lst[1][0] - lst[0][0]

        candidates.sort(key=lambda item: regret_score(item[2]), reverse=True)
        for (vid, alloc_vec, best_list) in candidates:
            if not best_list:
                remainders_for_next_round.append((vid, alloc_vec))
                continue
            delta_cost, r, delta_d, pos = best_list[0]
            if not feasible_add_chunk_to_route(r, alloc_vec):
                remainders_for_next_round.append((vid, alloc_vec))
                continue
            apply_insert_at(r, vid, alloc_vec, delta_d, pos)
            if vid in sol.vendor_remaining:
                for j in range(sol.n_goods):
                    sol.vendor_remaining[vid][j] -= alloc_vec[j]

        processing_list = remainders_for_next_round
    return True

# ======================== FEASIBILITY SWEEP ========================
def force_collect_all(sol: Solution, vehicles_json, dist_fn, DEPOT_NODE_TO_INTERNAL: Dict[int,int], GOODS: List[str]) -> None:
    if not sol.routes:
        return
    depot_node = sol.routes[0].depot
    depot_internal = DEPOT_NODE_TO_INTERNAL[depot_node]

    best_v, base_cap_vec, veh_cost, veh_idx = pick_cheapest_vehicle(vehicles_json, depot_internal, GOODS)
    if best_v is None:
        for vid, vec in sol.vendor_remaining.items():
            if sum(vec) > 0:
                sol.unassigned.add(vid)
                sol.vendor_remaining[vid] = [0]*sol.n_goods
        return
    
    veh_type = best_v.get("vehicle_type", "type_1") # <-- ADDED

    while True:
        todo = [(vid, vec) for vid, vec in sol.vendor_remaining.items() if sum(vec) > 0]
        if not todo:
            break
        vid, rem_vec = min(todo, key=lambda t: dist_fn(depot_node, t[0])) # gần depot hơn

        if dist_fn(depot_node, vid) >= HUGE or dist_fn(vid, depot_node) >= HUGE:
            sol.unassigned.add(vid)
            sol.vendor_remaining[vid] = [0]*sol.n_goods
            continue

        fit_vec, _ = fit_alloc_into_cap(rem_vec, base_cap_vec)
        if sum(fit_vec) <= 0:
            sol.unassigned.add(vid)
            sol.vendor_remaining[vid] = [0]*sol.n_goods
            continue

        new_r = Route.new(depot_node, veh_idx, veh_cost, base_cap_vec, GOODS, vehicle_type=veh_type) # <-- MODIFIED
        delta_d = delta_insert_edge(new_r, vid, dist_fn, insert_pos=1)
        if delta_d >= HUGE:
            sol.unassigned.add(vid)
            sol.vendor_remaining[vid] = [0]*sol.n_goods
            continue

        apply_insert_at(new_r, vid, fit_vec, delta_d, insert_pos=1)
        sol.routes.append(new_r)
        sub_vector_inplace(sol.vendor_remaining[vid], fit_vec)

# ======================== ALNS KHỞI TẠO & VÒNG LẶP ========================
def build_initial_solution_for_cluster(
    cluster,
    vehicles_json,
    vehicle_costs,
    dist_fn,
    demand_map,
    min_cap_scaled,
    DEPOT_NODE_TO_INTERNAL: Dict[int, int],
    GOODS: List[str]
) -> Solution:
    depot_node = int(cluster["depot_node_id"])
    depot_internal = DEPOT_NODE_TO_INTERNAL[depot_node]
    vendors = [int(v) for v in cluster["assigned_vendors"]]
    n_goods = len(GOODS)

    vehicles = vehicles_of_depot(vehicles_json, depot_internal)
    if not vehicles:
        print(f"Warning: Không có xe cho depot {depot_node} (internal {depot_internal}).")
        vendor_remaining = {}
        for vid in vendors:
            vec = [int(round(float(demand_map.get(vid, {}).get(g, 0.0))*SCALE)) for g in GOODS]
            vendor_remaining[vid] = vec
        sol = Solution(routes=[], unassigned=set(vendors), vendor_remaining=vendor_remaining, n_goods=n_goods, cost=0.0)
        return sol

    routes: List[Route] = []
    for vi, v in enumerate(vehicles):
        cap_vec = vehicle_capacity_vec(v, GOODS)
        veh_cost = vehicle_cost_per_km(v, vehicles_json["meta"]["vehicle_costs"])
        veh_type = v.get("vehicle_type", "type_1") # <-- ADDED
        routes.append(Route.new(depot_node, vi, veh_cost, cap_vec, GOODS, vehicle_type=veh_type)) # <-- MODIFIED

    per_caps_total = [sum(vehicle_capacity_vec(v, GOODS)) for v in vehicles]
    min_cap = int(max(1, floor(min(per_caps_total))))
    chunk_nodes, chunk_qty_scaled, _ = integer_split_scaled_safe(vendors, demand_map, min_cap_scaled=min_cap, GOODS=GOODS)

    vendor_remaining: Dict[int, List[int]] = {}
    for vid in vendors:
        vec = [int(round(float(demand_map.get(vid, {}).get(g, 0.0))*SCALE)) for g in GOODS]
        vendor_remaining[vid] = vec

    processing_list = list(zip(chunk_nodes, chunk_qty_scaled))

    while processing_list:
        (vid, qty_scaled) = processing_list.pop(0)
        alloc = allocate_chunk_vector(vendor_remaining[vid], qty_scaled, GOODS)
        if sum(alloc) <= 0:
            continue

        best = None
        for r in routes:
            if not feasible_add_chunk_to_route(r, alloc):
                continue
            for pos in range(1, len(r.path)):
                delta_d = delta_insert_edge(r, vid, dist_fn, pos)
                if delta_d >= HUGE:
                    continue
                delta_cost = delta_d * r.veh_cost
                if (best is None) or (delta_cost < best[0]):
                    best = (delta_cost, r, delta_d, pos)

        if best is not None:
            _, r, delta_d, pos = best
            apply_insert_at(r, vid, alloc, delta_d, pos)
            sub_vector_inplace(vendor_remaining[vid], alloc)
        else:
            best_v, base_cap_vec, veh_cost, veh_idx = pick_cheapest_vehicle(vehicles_json, depot_internal, GOODS)
            if best_v is None:
                continue
            
            veh_type = best_v.get("vehicle_type", "type_1") # <-- ADDED
            new_r = Route.new(depot_node, veh_idx, veh_cost, base_cap_vec, GOODS, vehicle_type=veh_type) # <-- MODIFIED
            
            fit_vec, rem_vec = fit_alloc_into_cap(alloc, new_r.cap_vec)
            if sum(fit_vec) <= 0:
                continue
            if dist_fn(depot_node, vid) >= HUGE or dist_fn(vid, depot_node) >= HUGE:
                continue
            delta_d = delta_insert_edge(new_r, vid, dist_fn, insert_pos=1)
            if delta_d < HUGE:
                apply_insert_at(new_r, vid, fit_vec, delta_d, insert_pos=1)
                sub_vector_inplace(vendor_remaining[vid], fit_vec)
                routes.append(new_r)
                if sum(rem_vec) > 0:
                    processing_list.insert(0, (vid, int(sum(rem_vec))))

    sol = Solution(routes, set(), vendor_remaining, n_goods)
    force_collect_all(sol, vehicles_json, dist_fn, DEPOT_NODE_TO_INTERNAL, GOODS)
    recompute_all_cost(sol, dist_fn)
    return sol

def roulette_select(weights: List[float]) -> int:
    s = sum(weights)
    if s <= 0:
        return int(random.random() * len(weights))
    threshold = random.random() * s
    acc = 0.0
    for i, w in enumerate(weights):
        acc += w
        if acc >= threshold:
            return i
    return len(weights)-1

def update_weights(weights: List[float], scores: List[float], counts: List[int], rho: float = RHO) -> None:
    for i in range(len(weights)):
        avg = scores[i] / max(1, counts[i])
        weights[i] = (1 - rho)*weights[i] + rho*avg
        scores[i] = 0.0
        counts[i] = 0

def alns_optimize_cluster(
    cluster,
    vehicles_json,
    dist_fn,
    demand_map,
    DEPOT_NODE_TO_INTERNAL: Dict[int, int],
    GOODS: List[str],
    max_iter=8000,
    destroy_ratio=0.25,
    cooling=0.997
):
    depot_node = int(cluster["depot_node_id"])
    print(f"\n--- Bắt đầu tối ưu cho Depot {depot_node} ---")
    depot_internal = DEPOT_NODE_TO_INTERNAL.get(depot_node)
    if depot_internal is None:
        print(f"LỖI: Depot ID {depot_node} không có trong map.")
        return {"depot_node": depot_node, "routes": [], "distance_cost_sum": 0.0, "unassigned_vendors": [int(v) for v in cluster["assigned_vendors"]]}

    vehicles = vehicles_of_depot(vehicles_json, depot_internal)
    if not vehicles:
        print(f"Info: Không có xe cho depot {depot_node}.")
        return {"depot_node": depot_node, "routes": [], "distance_cost_sum": 0.0, "unassigned_vendors": [int(v) for v in cluster["assigned_vendors"]]}

    per_caps_total = [sum(vehicle_capacity_vec(v, GOODS)) for v in vehicles]
    if not per_caps_total or min(per_caps_total) <= 0:
        print(f"LỖI: Xe của depot {depot_node} không có capacity.")
        return {"depot_node": depot_node, "routes": [], "distance_cost_sum": 0.0, "unassigned_vendors": [int(v) for v in cluster["assigned_vendors"]]}

    min_cap = int(max(1, floor(min(per_caps_total))))

    curr = build_initial_solution_for_cluster(
        cluster, vehicles_json, vehicles_json["meta"]["vehicle_costs"],
        dist_fn, demand_map, min_cap,
        DEPOT_NODE_TO_INTERNAL, GOODS
    )
    best = curr.clone()
    best_cost = recompute_all_cost(best, dist_fn)
    curr_cost = best_cost

    if best_cost >= HUGE:
        print(f"Info: Không thể tạo solution ban đầu cho depot {depot_node}. Chi phí: {best_cost}")
        return {"depot_node": depot_node, "routes": [], "distance_cost_sum": 0.0, "unassigned_vendors": list(best.unassigned)}

    print(f"Depot {depot_node}: Chi phí ban đầu = {best_cost:.2f} (Unreachable: {len(best.unassigned)})")

    destroy_ops = [
        ("random_removal", lambda s: random_removal(s, dist_fn, remove_ratio=destroy_ratio)),
        ("worst_removal",  lambda s: worst_removal(s, dist_fn, remove_ratio=destroy_ratio)),
        ("shaw_removal",   lambda s: shaw_removal(s, dist_fn, remove_ratio=destroy_ratio)),
    ]
    repair_ops = [
        ("greedy",   lambda s, rem: greedy_best_position_insertion(s, rem, dist_fn, GOODS, vehicles_json, DEPOT_NODE_TO_INTERNAL)),
        ("regret-2", lambda s, rem: regret2_best_position_insertion(s, rem, dist_fn, GOODS, vehicles_json, DEPOT_NODE_TO_INTERNAL)),
    ]
    w_d = [1.0]*len(destroy_ops)
    w_r = [1.0]*len(repair_ops)
    scr_d = [0.0]*len(destroy_ops)
    cnt_d = [0]*len(destroy_ops)
    scr_r = [0.0]*len(repair_ops)
    cnt_r = [0]*len(repair_ops)

    T = max(1.0, (best_cost % HUGE) * 0.05)
    iter_since_last_weight_update = 0
    best_iter = 0

    for it in range(max_iter):
        di = roulette_select(w_d)
        ri = roulette_select(w_r)

        trial = curr.clone()
        removed = destroy_ops[di][1](trial)
        if not removed:
            continue

        repair_ops[ri][1](trial, removed)
        new_cost = recompute_all_cost(trial, dist_fn)

        improved_curr = new_cost < curr_cost
        improved_best = new_cost < best_cost
        reward = REWARD_GLOBAL_BEST if improved_best else (REWARD_IMPROVED if improved_curr else REWARD_ACCEPTED)
        scr_d[di] += reward; cnt_d[di] += 1
        scr_r[ri] += reward; cnt_r[ri] += 1

        delta = new_cost - curr_cost
        accept = (delta < 0) or (random.random() < exp(-delta / max(T, 1e-9)))

        if accept:
            curr = trial
            curr_cost = new_cost
            if improved_best:
                best = trial.clone()
                best_cost = new_cost
                best_iter = it

        T *= cooling
        iter_since_last_weight_update += 1
        if iter_since_last_weight_update >= SEGMENT_LENGTH:
            update_weights(w_d, scr_d, cnt_d, rho=RHO)
            update_weights(w_r, scr_r, cnt_r, rho=RHO)
            iter_since_last_weight_update = 0

        if it - best_iter >= EARLY_STOP_PATIENCE:
            print(f"Depot {depot_node}: Dừng sớm tại iter {it} (không cải thiện từ iter {best_iter})")
            break

    force_collect_all(best, vehicles_json, dist_fn, DEPOT_NODE_TO_INTERNAL, GOODS)
    best_cost = recompute_all_cost(best, dist_fn)

    # ====== EXPAND PATH Ở OUTPUT (THEO YÊU CẦU A) ======
    # Sử dụng path_map toàn cục do main() cung cấp thông qua closure; ở đây ta chỉ build routes_out theo route.path
    # Hàm expand_path sẽ được set trong main.
    routes_out, total = [], 0.0
    for r in best.routes:
        d = r.dist_cache
        if d <= 0 or d >= HUGE:
            continue
        dist_cost = d * r.veh_cost
        routes_out.append({
            "vehicle_id": f"depot{DEPOT_NODE_TO_INTERNAL[depot_node]}_veh{r.vehicle_idx}",
            "vehicle_cost_per_km": r.veh_cost,
            "vehicle_type": r.vehicle_type, # <-- ADDED
            # 'path_node_ids' sẽ expand tại main() trước khi ghi file out cuối cùng
            "path_node_ids": r.path[:],
            "distance_raw": float(d),
            "route_cost": float(dist_cost)
        })
        total += dist_cost

    if best.unassigned:
        print(f"⚠️ Depot {depot_node}: Không thể gom (unreachable) {len(best.unassigned)} vendors: {sorted(list(best.unassigned))}")

    print(f"Depot {depot_node}: Hoàn thành. Chi phí tốt nhất = {total:.2f} (Unreachable: {len(best.unassigned)})")
    return {"depot_node": depot_node, "routes": routes_out, "distance_cost_sum": total, "unassigned_vendors": list(best.unassigned)}

# ======================== EXPAND FULL PATH CHO OUTPUT ========================
def expand_route_path(path_ids: List[int], path_map: Dict[Tuple[int,int], List[int]]) -> List[int]:
    """
    Biến [a, v1, v2, ..., depot] thành chuỗi node đầy đủ bằng cách thay mỗi cạnh (u->v)
    bằng path_map[u,v] nếu có (bỏ lặp u), ngược lại giữ (u,v) như cũ.
    """
    if not path_ids or len(path_ids) == 1:
        return path_ids[:]
    out = [path_ids[0]]
    for i in range(len(path_ids)-1):
        u, v = path_ids[i], path_ids[i+1]
        key = (u, v)
        if key in path_map and len(path_map[key]) >= 2:
            seg = path_map[key]
            # thêm toàn bộ trừ node đầu (tránh lặp)
            out.extend(seg[1:])
        else:
            # không có path_map -> giữ cạnh trực tiếp
            out.append(v)
    return out

def main():
    parser = argparse.ArgumentParser(description="ALNS Pickup + On-demand shortest-path for blocked depot<->vendor edges (expand path at output)")
    parser.add_argument("--base_dir", type=str, default=r"C:\Users\thaip\Desktop\VRP4\data\JSON\osaba-50-instances")
    parser.add_argument("--cluster_file", type=str, default="vendor_cluster_50.json")
    parser.add_argument("--nodes_file", type=str, default="osaba_50_standard.json")
    parser.add_argument("--matrix_file", type=str, default="osaba_50_graph_matrix.json")
    parser.add_argument("--vehicles_file", type=str, default="osaba_50_vehicles.json")
    parser.add_argument("--max_iter", type=int, default=2000)
    parser.add_argument("--destroy_ratio", type=float, default=0.18)
    parser.add_argument("--cooling", type=float, default=0.998)
    args = parser.parse_args() if 'get_ipython' not in globals() else argparse.Namespace(
        base_dir=r"C:\Users\thaip\Desktop\VRP4\data\JSON\osaba-50-instances",
        cluster_file="vendor_cluster_50.json",
        nodes_file="osaba_50_standard.json",
        matrix_file="osaba_50_graph_matrix.json",
        vehicles_file="osaba_50_vehicles.json",
        max_iter=2000,
        destroy_ratio=0.18,
        cooling=0.998
    )

    random.seed(SEED)

    base_dir = Path(args.base_dir)
    path_cluster = base_dir / args.cluster_file
    path_nodes = base_dir / args.nodes_file
    path_matrix = base_dir / args.matrix_file
    path_vehicles = base_dir / args.vehicles_file

    print("Đang tải file...")
    nodes = load_json(path_nodes)
    clusters = load_json(path_cluster)
    vehicles = load_json(path_vehicles)
    dist_json = load_json(path_matrix)

    GOODS = nodes["meta"]["commodities"]
    depot_nodes = [int(n["id"]) for n in nodes["nodes"] if n["type"] == "depot"]
    depot_nodes.sort()
    DEPOT_NODE_TO_INTERNAL = {node_id: i for i, node_id in enumerate(depot_nodes)}

    floyd_path = path_matrix.with_name(path_matrix.stem + "_floyd.json")
    dist_num, path_map = preprocess_only_for_blocked_pairs(nodes, clusters, dist_json, floyd_path)
    id2idx = build_id_to_idx(nodes)
    dist_fn = dist_accessor(dist_num, id2idx, path_map)
    demand = get_vendor_demand(nodes, GOODS)

    total_cost = 0.0
    results, total_unassigned = [], []

    for c in clusters["clusters"]:
        res = alns_optimize_cluster(
            c, vehicles_json=vehicles, dist_fn=dist_fn,
            demand_map=demand, DEPOT_NODE_TO_INTERNAL=DEPOT_NODE_TO_INTERNAL,
            GOODS=GOODS, max_iter=args.max_iter, destroy_ratio=args.destroy_ratio,
            cooling=args.cooling
        )
        results.append(res)
        total_cost += float(res.get("distance_cost_sum", 0.0))
        if res.get("unassigned_vendors"):
            total_unassigned.extend(res["unassigned_vendors"])

    # ===== Expand & GOM THEO vehicle_type =====
    print("\n============ SUMMARY VEHICLE TYPE ROUTES ============")
    for cluster_res in results:
        depot = cluster_res["depot_node"]
        routes = cluster_res["routes"]
        print(f"\nDepot {depot}:")
        type_summary = {}

        for r in routes:
            # Expand đường nếu có path_map
            r["path_node_ids"] = expand_route_path(r["path_node_ids"], path_map)
            
            # --- MODIFIED BLOCK ---
            # Lấy loại xe trực tiếp từ kết quả (đã được thêm vào ở alns_optimize_cluster)
            veh_type = r.get("vehicle_type", "unknown")
            # --- END MODIFIED BLOCK ---
            
            # Gom nhóm
            if veh_type not in type_summary:
                type_summary[veh_type] = []
            type_summary[veh_type].append(r["path_node_ids"])

        # In console
        for vt, trips in type_summary.items():
            print(f"  {vt}:")
            for i, trip in enumerate(trips, 1):
                path_str = " → ".join(map(str, trip))
                print(f"    Trip {i} → {path_str}")

        cluster_res["vehicle_type_summary"] = type_summary  # thêm vào output JSON

    # ===== Lưu OUTPUT =====
    output_path = path_cluster.parent / f"alns_routes_pickup.json"
    out = {
        "objective": "sum(distance * vehicle_type_cost) over routes (Pickup Phase)",
        "total_cost": total_cost,
        "distance_unit": "raw_from_matrix",
        "seed": SEED,
        "clusters": results,
        "total_unassigned_vendors": sorted(list(set(total_unassigned))),
        "preprocessed_graph": str(floyd_path.name)
    }
    save_json(output_path, out)

    print("\n" + "="*50)
    print(f"✅ Đã lưu kết quả ALNS pickup vào: {output_path}")
    print(f"Tổng chi phí (Tất cả depot): {total_cost:.2f}")
    if total_unassigned:
        print(f"⚠️ Không gom được {len(set(total_unassigned))} vendors: {sorted(list(set(total_unassigned)))}")

if __name__ == "__main__":
    main()

Đang tải file...

--- Bắt đầu tối ưu cho Depot 6 ---
Depot 6: Chi phí ban đầu = 16238.40 (Unreachable: 0)
Depot 6: Hoàn thành. Chi phí tốt nhất = 16238.40 (Unreachable: 0)

--- Bắt đầu tối ưu cho Depot 16 ---
Depot 16: Chi phí ban đầu = 16024.20 (Unreachable: 0)
Depot 16: Hoàn thành. Chi phí tốt nhất = 16024.20 (Unreachable: 0)

--- Bắt đầu tối ưu cho Depot 25 ---
Depot 25: Chi phí ban đầu = 15526.40 (Unreachable: 0)
Depot 25: Hoàn thành. Chi phí tốt nhất = 15526.40 (Unreachable: 0)

--- Bắt đầu tối ưu cho Depot 36 ---
Depot 36: Chi phí ban đầu = 18345.80 (Unreachable: 0)
Depot 36: Hoàn thành. Chi phí tốt nhất = 18345.80 (Unreachable: 0)

--- Bắt đầu tối ưu cho Depot 45 ---
Depot 45: Chi phí ban đầu = 16302.20 (Unreachable: 0)
Depot 45: Hoàn thành. Chi phí tốt nhất = 14700.60 (Unreachable: 0)

============ SUMMARY VEHICLE TYPE ROUTES ============

Depot 6:
  medium_truck:
    Trip 1 → 6 → 2 → 2 → 6
  small_truck:
    Trip 1 → 6 → 2 → 6
    Trip 2 → 6 → 5 → 6
    Trip 3 → 6 → 7 → 8 → 7 

In [3]:
# -*- coding: utf-8 -*-
import json
import random
import re
import heapq
from math import ceil, floor, exp
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Callable, Any # <-- ĐÃ THÊM 'List, Dict, ...'
import argparse
import sys

# ======================== CẤU HÌNH CHUNG ========================
SEED = 42
random.seed(SEED)

SCALE = 1000            # scale cho khối lượng
CHUNK_SAFETY = 0.95     # cắt chunk <= 95% min cap một trip
HUGE = 1_000_000_000

# ALNS
SEGMENT_LENGTH = 50
RHO = 0.6
REWARD_GLOBAL_BEST = 6.0
REWARD_IMPROVED = 3.0
REWARD_ACCEPTED = 1.0
EARLY_STOP_PATIENCE = 2000

# ======================== TIỆN ÍCH DATA ========================
def load_json(path: Path) -> Any:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def save_json(path: Path, obj: Any) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

def build_id_to_idx(nodes_json) -> Dict[int, int]:
    return {int(n["id"]): idx for idx, n in enumerate(nodes_json["nodes"])}

# <-- MARKET MOD: Renamed function and changed type check
def get_market_demand(nodes_json, GOODS: List[str]) -> Dict[int, Dict[str, float]]:
    out = {}
    for n in nodes_json["nodes"]:
        # <-- MARKET MOD: Check for "market" type
        if n.get("type") != "market": 
            continue
        vid = int(n["id"])
        gdict = (
            n.get("order_dict") # Ưu tiên order_dict cho market
            or n.get("goods_dict")
            or n.get("collected_goods_dict")
            or n.get("collected_goods")
            or {}
        )
        out[vid] = {g: float(gdict.get(g, 0.0)) for g in GOODS}
    return out

# ======================== XE & CHI PHÍ ========================
def vehicles_of_depot(vehicles_json, depot_internal: int) -> List[dict]:
    return [v for v in vehicles_json["vehicles"] if int(v["depot_id"]) == depot_internal]

def vehicle_capacity_vec(v: dict, GOODS: List[str]) -> List[int]:
    return [int(round(float(v["capacity_dict"][g]) * SCALE)) for g in GOODS]

def vehicle_cost_per_km(v: dict, vehicle_costs_table: Dict[str, float]) -> float:
    t = v.get("vehicle_type", "type_1")
    return float(vehicle_costs_table.get(t, 1.0))

def _parse_vehicle_idx_as_int(v: dict) -> int:
    raw = str(v.get("vehicle_idx", v.get("vehicle_id", "0")))
    m = re.search(r'\d+', raw)
    return int(m.group()) if m else 0

def pick_cheapest_vehicle(vehicles_json, depot_internal: int, GOODS: List[str]):
    vehs = vehicles_of_depot(vehicles_json, depot_internal)
    if not vehs:
        return None, None, None, None
    costs = vehicles_json["meta"]["vehicle_costs"]
    def _cost(v): return vehicle_cost_per_km(v, costs)
    best = min(vehs, key=_cost)
    cap_vec = vehicle_capacity_vec(best, GOODS)
    veh_cost = _cost(best)
    veh_idx = _parse_vehicle_idx_as_int(best)
    return best, cap_vec, veh_cost, veh_idx

# ======================== TOÁN VECTORS ========================
def sub_vector_inplace(a: List[int], b: List[int]) -> None:
    for i in range(len(a)): a[i] -= b[i]

def add_vector_inplace(a: List[int], b: List[int]) -> None:
    for i in range(len(a)): a[i] += b[i]

def leq_vec(a: List[int], b: List[int]) -> bool:
    return all(a[i] <= b[i] for i in range(len(a)))

def fit_alloc_into_cap(alloc_vec: List[int], cap_vec: List[int]) -> Tuple[List[int], List[int]]:
    fit = [min(alloc_vec[i], cap_vec[i]) for i in range(len(alloc_vec))]
    rem = [alloc_vec[i] - fit[i] for i in range(len(alloc_vec))]
    return fit, rem

# ======================== DỮ LIỆU GRAPH & RÚT GỌN CẠNH CẤM ========================
def _norm_val(x) -> float:
    if x is None: return float("inf")
    if isinstance(x, str) and x.lower() == "infinity": return float("inf")
    return float(x)

def build_graph_from_matrix(distance_matrix: List[List[Any]]) -> List[List[float]]:
    n = len(distance_matrix)
    G = [[float("inf")] * n for _ in range(n)]
    for i in range(n):
        G[i][i] = 0.0
        row = distance_matrix[i]
        for j in range(n):
            val = _norm_val(row[j])
            if val < float("inf"):
                G[i][j] = val
    return G

def dijkstra_path(G: List[List[float]], src: int, dst: int) -> Tuple[float, List[int]]:
    n = len(G)
    dist = [float("inf")] * n
    parent = [-1] * n
    dist[src] = 0.0
    pq = [(0.0, src)]
    while pq:
        d, u = heapq.heappop(pq)
        if d != dist[u]: 
            continue
        if u == dst:
            break
        for v in range(n):
            w = G[u][v]
            if w == float("inf"): 
                continue
            nd = d + w
            if nd < dist[v]:
                dist[v] = nd
                parent[v] = u
                heapq.heappush(pq, (nd, v))
    if dist[dst] == float("inf"):
        return float("inf"), []
    # reconstruct path (indices)
    path = []
    cur = dst
    while cur != -1:
        path.append(cur)
        cur = parent[cur]
    path.reverse()
    return dist[dst], path

def preprocess_only_for_blocked_pairs(
    nodes_json, clusters_json, matrix_json, output_path: Path
) -> Tuple[List[List[float]], Dict[Tuple[int,int], List[int]]]:
    """
    - Nhận matrix gốc (chứa 'Infinity' / None cho cạnh cấm).
    - Chỉ rút gọn (tìm đường đi vòng) cho các cặp 'depot<->market' bị cấm trực tiếp. # <-- MARKET MOD
    - Trả về:
        + dist_apsp: ma trận distance mới (chỉ cập nhật ở các cặp đã chữa),
        + path_map: map (node_id_u, node_id_v) -> [id_u, ..., id_v] là đường đi đầy đủ.
    - Đồng thời xuất file JSON '..._floyd.json' như yêu cầu.
    """
    distance_matrix = matrix_json["distance_matrix"]
    id_to_idx = build_id_to_idx(nodes_json)
    idx_to_id = {idx: nid for nid, idx in id_to_idx.items()}
    G = build_graph_from_matrix(distance_matrix)  # adjacency-weighted directed graph

    # Lấy danh sách depot & market
    depots = [int(n["id"]) for n in nodes_json["nodes"] if n.get("type") == "depot"]
    # <-- MARKET MOD: Find "market" nodes
    markets = [int(n["id"]) for n in nodes_json["nodes"] if n.get("type") == "market"]

    # Chuẩn bị dist & path_map
    n = len(distance_matrix)
    dist_out = [[_norm_val(distance_matrix[i][j]) for j in range(n)] for i in range(n)]
    path_map: Dict[Tuple[int,int], List[int]] = {}

    # Chỉ xét cặp depot<->market thuộc cluster assignment (filter nhẹ nhàng)
    assigned = set()
    
    # --- START ROBUST FIX ---
    # (Đoạn này đã được sửa để xử lý file cluster không có key "clusters")
    cluster_list = clusters_json.get("clusters")
    if cluster_list is None:
        if isinstance(clusters_json, list):
            cluster_list = clusters_json
        else:
            # Nếu vẫn không tìm thấy, thử tìm key đầu tiên là list
            found_list = [v for v in clusters_json.values() if isinstance(v, list)]
            if found_list:
                cluster_list = found_list[0]
            else:
                # Fallback, có thể thất bại nếu file JSON thực sự trống
                cluster_list = [] 
                print(f"CẢNH BÁO: Không tìm thấy key 'clusters' hoặc một list cluster nào trong file {output_path.name}")
    # --- END ROBUST FIX ---

    for c in cluster_list:
        # <-- MARKET MOD: Key might be "assigned_markets" or "assigned_vendors"
        key_to_use = "assigned_markets" if "assigned_markets" in c else "assigned_vendors"
        if key_to_use not in c: continue # Bỏ qua nếu cluster không có key
            
        for v in c[key_to_use]:
            assigned.add(int(v))
    
    markets = [v for v in markets if v in assigned]

    # Chữa chiều đi & về khi bị chặn trực tiếp (Infinity)
    for depot_id in depots:
        for market_id in markets: # <-- MARKET MOD
            if depot_id not in id_to_idx or market_id not in id_to_idx:
                continue # Bỏ qua nếu node ID không có trong id_to_idx
            
            iu = id_to_idx[depot_id]; iv = id_to_idx[market_id] # <-- MARKET MOD
            
            # depot -> market
            if dist_out[iu][iv] == float("inf"):
                d, p_idx = dijkstra_path(G, iu, iv)
                if d < float("inf"):
                    dist_out[iu][iv] = d
                    # lưu full path theo ID
                    path_map[(depot_id, market_id)] = [idx_to_id[k] for k in p_idx]
            # market -> depot
            if dist_out[iv][iu] == float("inf"):
                d, p_idx = dijkstra_path(G, iv, iu)
                if d < float("inf"):
                    dist_out[iv][iu] = d
                    path_map[(market_id, depot_id)] = [idx_to_id[k] for k in p_idx]

    # Xuất file JSON mới
    dist_out_json = {
        "distance_matrix": [
            [("Infinity" if x == float("inf") else x) for x in row]
            for row in dist_out
        ],
        "path_reconstruction": {
            f"{u}_{v}": path_map[(u,v)] for (u,v) in path_map
        }
    }
    save_json(output_path, dist_out_json)
    return dist_out, path_map

# ======================== DIST ACCESSOR (DÙNG CẢ PATH_MAP) ========================
def dist_accessor(distance_matrix_num, id_to_idx, path_map: Dict[Tuple[int,int], List[int]]) -> Callable[[int,int], float]:
    """
    Nếu (u,v) có path_map -> trả tổng khoảng cách theo path (đảm bảo finite).
    Nếu không, đọc trực tiếp từ matrix (có thể Infinity).
    """
    n = len(distance_matrix_num)

    # tiền tính chiều dài đường đi từ path_map để nhanh hơn
    cache_cost: Dict[Tuple[int,int], float] = {}
    def pair_cost(u: int, v: int) -> float:
        key = (u, v)
        if key in cache_cost:
            return cache_cost[key]
        if key in path_map:
            path_ids = path_map[key]
            # tính tổng theo matrix_num (đã là trọng số cạnh trực tiếp)
            total = 0.0
            for i in range(len(path_ids)-1):
                a = id_to_idx.get(int(path_ids[i]), None)
                b = id_to_idx.get(int(path_ids[i+1]), None)
                if a is None or b is None: 
                    cache_cost[key] = float("inf"); return cache_cost[key]
                w = distance_matrix_num[a][b]
                if w == float("inf"):
                    cache_cost[key] = float("inf"); return cache_cost[key]
                total += w
            cache_cost[key] = total
            return total
        
        # nếu không có path_map, lấy ô trực tiếp
        ia = id_to_idx.get(int(u), None)
        ib = id_to_idx.get(int(v), None)
        if ia is None or ib is None: 
            return float("inf")
        val = distance_matrix_num[ia][ib]
        return val

    def d(a: int, b: int) -> float:
        val = pair_cost(a, b)
        if val == float("inf"):
            return HUGE
        return float(val)

    return d

# ======================== CHUNKING ========================
def integer_split_scaled_safe(
    nodes: List[int], # <-- MARKET MOD: Renamed vendors -> nodes
    demand_map: Dict[int, Dict[str, float]],
    min_cap_scaled: int,
    GOODS: List[str]
):
    if min_cap_scaled <= 0:
        min_cap_scaled = 1
    limit = int(max(1, CHUNK_SAFETY * min_cap_scaled))
    chunks_nodes: List[int] = []
    chunks_qty_scaled: List[int] = []
    for node_id in nodes: # <-- MARKET MOD: Renamed vid -> node_id
        total = float(sum(demand_map.get(node_id, {}).get(g, 0.0) for g in GOODS))
        total_scaled = int(round(total * SCALE))
        if total_scaled <= 0:
            continue
        
        # Tránh lỗi chia cho 0 nếu limit = 0
        if limit == 0:
            need = 1
        else:
            need = ceil(total_scaled / limit)
            
        if need == 0:
            need = 1 # Phải có ít nhất 1 chunk
            
        base = total_scaled // need
        rem  = total_scaled % need
        
        for i in range(int(need)):
            take = base + (1 if i < rem else 0)
            take = min(take, limit) # Đảm bảo không vượt quá giới hạn
            take = max(1, take)     # Đảm bảo lấy ít nhất 1
            chunks_nodes.append(node_id)
            chunks_qty_scaled.append(int(take))
            
    return chunks_nodes, chunks_qty_scaled, None

def allocate_chunk_vector(node_remaining_vec: List[int], qty_scaled: int, GOODS: List[str]) -> List[int]: # <-- MARKET MOD
    total_rem = sum(max(0, x) for x in node_remaining_vec)
    if total_rem <= 0:
        # Nếu không còn hàng, nhưng vẫn được yêu cầu phân bổ (ví dụ: chunk cuối cùng)
        # Trả về một vector 0
        return [0]*len(GOODS)
        
    alloc = []
    for rem_i in node_remaining_vec:
        portion = (rem_i / total_rem) if total_rem > 0 else 0.0
        part = int(round(qty_scaled * portion))
        alloc.append(part)
        
    diff = qty_scaled - sum(alloc)
    
    # Sửa logic phân bổ 'diff' để tránh vòng lặp vô tận nếu len(alloc) = 0
    if not alloc:
        return [0]*len(GOODS)

    i = 0
    while diff != 0 and i < len(alloc) * 2: # Thêm giới hạn vòng lặp
        idx = i % len(alloc)
        if diff > 0:
            # Chỉ thêm nếu chưa vượt quá
            if alloc[idx] < node_remaining_vec[idx]:
                alloc[idx] += 1; diff -= 1
        else: # diff < 0
            # Chỉ trừ nếu còn
            if alloc[idx] > 0:
                alloc[idx] -= 1; diff += 1
        i += 1

    # Đảm bảo alloc không vượt quá remaining_vec
    for i in range(len(alloc)):
        if alloc[i] > node_remaining_vec[i]:
            alloc[i] = node_remaining_vec[i]
        if alloc[i] < 0:
            alloc[i] = 0
            
    return alloc

# ======================== CẤU TRÚC LỜI GẢI ========================
@dataclass
class Route:
    depot: int
    vehicle_idx: int
    veh_cost: float
    cap_vec: List[int]
    n_goods: int
    vehicle_type: str = "unknown"
    path: List[int] = field(default_factory=list) # [depot, ..., depot] (chỉ market nodes) # <-- MARKET MOD
    load_vec: List[int] = field(default_factory=list)
    assigned_chunks: List[Dict[str, Any]] = field(default_factory=list)
    dist_cache: float = 0.0

    @staticmethod
    def new(depot_node_id: int, vehicle_idx: int, veh_cost_per_km: float, cap_vec: List[int], GOODS: List[str], vehicle_type: str = "unknown") -> "Route":
        n_goods = len(GOODS)
        return Route(
            depot=int(depot_node_id),
            vehicle_idx=vehicle_idx,
            veh_cost=float(veh_cost_per_km),
            cap_vec=cap_vec[:],
            n_goods=n_goods,
            vehicle_type=vehicle_type,
            path=[int(depot_node_id), int(depot_node_id)],
            load_vec=[0]*n_goods,
            assigned_chunks=[],
            dist_cache=0.0
        )

    def clone_shallow(self) -> "Route":
        return Route(
            depot=self.depot,
            vehicle_idx=self.vehicle_idx,
            veh_cost=self.veh_cost,
            cap_vec=self.cap_vec[:],
            n_goods=self.n_goods,
            vehicle_type=self.vehicle_type,
            path=self.path[:],
            load_vec=self.load_vec[:],
            # <-- MARKET MOD: Renamed vendor_id -> node_id
            assigned_chunks=[{"node_id": c.get("node_id", c.get("vendor_id")), "alloc": c["alloc"][:]} for c in self.assigned_chunks],
            dist_cache=self.dist_cache
        )

@dataclass
class Solution:
    routes: List[Route]
    unassigned: set              # chỉ chứa "unreachable thật"
    # <-- MARKET MOD: Renamed
    market_remaining: Dict[int, List[int]] 
    n_goods: int
    cost: float = 0.0

    def clone(self) -> "Solution":
        return Solution(
            routes=[r.clone_shallow() for r in self.routes],
            unassigned=set(self.unassigned),
            # <-- MARKET MOD: Renamed
            market_remaining={mid: vec[:] for mid, vec in self.market_remaining.items()},
            n_goods=self.n_goods,
            cost=self.cost
        )

# ======================== HÀM CHI PHÍ ========================
def compute_route_distance(route: Route, dist_fn: Callable[[int,int], float]) -> float:
    total = 0.0
    p = route.path
    for i in range(len(p)-1):
        d = dist_fn(p[i], p[i+1])
        if d >= HUGE:
            return float(HUGE)
        total += d
    return total

def recompute_all_cost(sol: Solution, dist_fn: Callable[[int,int], float]) -> float:
    total = 0.0
    for r in sol.routes:
        d = compute_route_distance(r, dist_fn)
        if d >= HUGE:
            sol.cost = float(HUGE)
            return float(HUGE)
        r.dist_cache = d
        total += d * r.veh_cost
    sol.cost = total
    return total

# ======================== TOÁN INSERT ========================
def delta_insert_edge(route: Route, node_id: int, dist_fn: Callable[[int,int], float], insert_pos: int) -> float: # <-- MARKET MOD
    p = route.path
    u = p[insert_pos-1]
    v = p[insert_pos]
    duv = dist_fn(u, v)
    dux = dist_fn(u, node_id) # <-- MARKET MOD
    dxv = dist_fn(node_id, v) # <-- MARKET MOD
    if dux >= HUGE or dxv >= HUGE or duv >= HUGE:
        return float(HUGE)
    return (dux + dxv - duv)

def apply_insert_at(route: Route, node_id: int, alloc_vec: List[int], delta_d: float, insert_pos: int) -> None: # <-- MARKET MOD
    route.path.insert(insert_pos, node_id)
    add_vector_inplace(route.load_vec, alloc_vec)
    # <-- MARKET MOD: Renamed vendor_id -> node_id
    route.assigned_chunks.append({"node_id": node_id, "alloc": alloc_vec[:]})
    route.dist_cache += float(delta_d)

def feasible_add_chunk_to_route(route: Route, alloc_vec: List[int]) -> bool:
    # Logic is the same: total deliveries must fit in the truck
    new_load = [route.load_vec[i] + alloc_vec[i] for i in range(route.n_goods)]
    return leq_vec(new_load, route.cap_vec)

# ======================== LẤY DANH SÁCH VISITS ========================
def list_all_visits(sol: Solution) -> List[Tuple[int,int,int]]:
    locs: List[Tuple[int,int,int]] = []
    for ri, r in enumerate(sol.routes):
        for pos in range(1, len(r.path)-1):
            node_id = r.path[pos] # <-- MARKET MOD
            if node_id != r.depot:
                locs.append((ri, pos, node_id))
    return locs

def remove_visit(sol: Solution, ri: int, pos: int) -> Tuple[int, List[int]]:
    r = sol.routes[ri]
    node_id = r.path[pos] # <-- MARKET MOD
    alloc = None
    for i in reversed(range(len(r.assigned_chunks))):
        # <-- MARKET MOD: Check for "node_id" or "vendor_id" for compatibility
        chunk_node_id = r.assigned_chunks[i].get("node_id", r.assigned_chunks[i].get("vendor_id"))
        if chunk_node_id == node_id:
            alloc = r.assigned_chunks.pop(i)["alloc"]
            break
    if alloc is None:
        alloc = [0]*sol.n_goods

    for j in range(sol.n_goods):
        r.load_vec[j] -= alloc[j]
        # <-- MARKET MOD: Renamed
        if node_id in sol.market_remaining:
            sol.market_remaining[node_id][j] += alloc[j]

    r.path.pop(pos)
    sol.unassigned.discard(node_id)
    return (node_id, alloc) # <-- MARKET MOD

# ======================== DESTROY OPERATORS ========================
def random_removal(sol: Solution, dist_fn, remove_ratio=0.15):
    locs = list_all_visits(sol)
    if not locs:
        return []
    k = max(1, int(len(locs)*remove_ratio))
    picked = random.sample(locs, k)
    picked.sort(key=lambda t: (t[0], t[1]), reverse=True)
    removed = []
    for (ri, pos, node_id) in picked: # <-- MARKET MOD
        removed.append(remove_visit(sol, ri, pos))
    return removed

def worst_removal(sol: Solution, dist_fn, remove_ratio=0.15):
    locs = list_all_visits(sol)
    if not locs:
        return []
    contribs = []
    for (ri, pos, node_id) in locs: # <-- MARKET MOD
        r = sol.routes[ri]
        u = r.path[pos-1]; x = r.path[pos]; v = r.path[pos+1]
        duv = dist_fn(u, v)
        c = (dist_fn(u, x) + dist_fn(x, v) - duv) * r.veh_cost
        contribs.append(((ri, pos, node_id), c)) # <-- MARKET MOD
    contribs.sort(key=lambda t: t[1], reverse=True)
    k = max(1, int(len(locs)*remove_ratio))
    picked_idx = [it[0] for it in contribs[:k]]
    picked_idx.sort(key=lambda t: (t[0], t[1]), reverse=True)
    removed = []
    for (ri, pos, node_id) in picked_idx: # <-- MARKET MOD
        removed.append(remove_visit(sol, ri, pos))
    return removed

def shaw_removal(sol: Solution, dist_fn, remove_ratio=0.15):
    locs = list_all_visits(sol)
    if not locs:
        return []
    k = max(1, int(len(locs)*remove_ratio))
    seed_ri, seed_pos, seed_node_id = random.choice(locs) # <-- MARKET MOD
    cand = []
    for (ri, pos, node_id) in locs: # <-- MARKET MOD
        if (ri, pos, node_id) == (seed_ri, seed_pos, seed_node_id):
            continue
        dist = dist_fn(seed_node_id, node_id) # <-- MARKET MOD
        cand.append(((ri, pos, node_id), dist))
    cand.sort(key=lambda t: t[1])
    picked_idx = [(seed_ri, seed_pos, seed_node_id)] + [c[0] for c in cand[:k-1]] # <-- MARKET MOD
    picked_idx.sort(key=lambda t: (t[0], t[1]), reverse=True)
    removed = []
    for (ri, pos, node_id) in picked_idx: # <-- MARKET MOD
        removed.append(remove_visit(sol, ri, pos))
    return removed

# ======================== HỖ TRỢ MỞ TRIP MỚI ========================
def _open_new_trip_and_fit(sol: Solution, node_id: int, alloc_vec: List[int], # <-- MARKET MOD
                           vehicles_json, DEPOT_NODE_TO_INTERNAL, dist_fn, GOODS: List[str]) -> Tuple[bool, List[int]]:
    if not sol.routes:
        # Nếu chưa có route nào, ta cần tạo 1 route
        # Tìm depot_node từ sol... 
        # Cách đơn giản: Yêu cầu sol phải có ít nhất 1 route (thường là route rỗng)
        # Hoặc: không thể mở trip mới nếu không biết depot
        return False, alloc_vec # Giả định: không thể mở trip mới nếu chưa có route nào
        
    depot_node = sol.routes[0].depot
    if depot_node not in DEPOT_NODE_TO_INTERNAL:
         print(f"LỖI: Depot node {depot_node} không có trong map DEPOT_NODE_TO_INTERNAL.")
         return False, alloc_vec
         
    depot_internal = DEPOT_NODE_TO_INTERNAL[depot_node]

    best_v, base_cap_vec, veh_cost, veh_idx = pick_cheapest_vehicle(vehicles_json, depot_internal, GOODS)
    if best_v is None:
        return False, alloc_vec
    
    veh_type = best_v.get("vehicle_type", "type_1")

    fit_vec, rem_vec = fit_alloc_into_cap(alloc_vec, base_cap_vec)
    if sum(fit_vec) <= 0:
        return False, alloc_vec

    new_r = Route.new(depot_node, veh_idx, veh_cost, base_cap_vec, GOODS, vehicle_type=veh_type)

    if dist_fn(depot_node, node_id) >= HUGE or dist_fn(node_id, depot_node) >= HUGE: # <-- MARKET MOD
        return False, alloc_vec

    delta_d = delta_insert_edge(new_r, node_id, dist_fn, insert_pos=1) # <-- MARKET MOD
    if delta_d >= HUGE:
        return False, alloc_vec

    apply_insert_at(new_r, node_id, fit_vec, delta_d, insert_pos=1) # <-- MARKET MOD
    sol.routes.append(new_r)

    # <-- MARKET MOD: Renamed
    if node_id in sol.market_remaining:
        for j in range(sol.n_goods):
            sol.market_remaining[node_id][j] -= fit_vec[j]

    return (sum(rem_vec) == 0), rem_vec

# ======================== REPAIR OPERATORS (FEASIBLE-BẰNG-MỌI-GIÁ) ========================
def greedy_best_position_insertion(sol: Solution, removed_list, dist_fn, GOODS: List[str],
                                   vehicles_json=None, DEPOT_NODE_TO_INTERNAL=None) -> bool:
    processing_list = removed_list[:]
    while processing_list:
        (node_id, alloc_vec) = processing_list.pop(0) # <-- MARKET MOD
        if sum(alloc_vec) <= 0:
            continue
        best = None
        for r in sol.routes:
            if not feasible_add_chunk_to_route(r, alloc_vec):
                continue
            for pos in range(1, len(r.path)):
                delta_d = delta_insert_edge(r, node_id, dist_fn, pos) # <-- MARKET MOD
                if delta_d >= HUGE:
                    continue
                delta_cost = delta_d * r.veh_cost
                if (best is None) or (delta_cost < best[0]):
                    best = (delta_cost, r, delta_d, pos)

        if best is not None:
            _, r, delta_d, pos = best
            apply_insert_at(r, node_id, alloc_vec, delta_d, pos) # <-- MARKET MOD
            # <-- MARKET MOD: Renamed
            if node_id in sol.market_remaining:
                for j in range(sol.n_goods):
                    sol.market_remaining[node_id][j] -= alloc_vec[j]
        else:
            # Chỉ mở trip mới nếu có thông tin vehicle
            if vehicles_json and DEPOT_NODE_TO_INTERNAL:
                done, rem_vec = _open_new_trip_and_fit(sol, node_id, alloc_vec, vehicles_json, DEPOT_NODE_TO_INTERNAL, dist_fn, GOODS) # <-- MARKET MOD
                if not done and sum(rem_vec) > 0:
                    processing_list.insert(0, (node_id, rem_vec)) # <-- MARKET MOD
            else:
                # Không thể mở trip mới, gán lại vào unassigned
                if node_id not in sol.market_remaining:
                     sol.market_remaining[node_id] = [0] * sol.n_goods
                add_vector_inplace(sol.market_remaining[node_id], alloc_vec)
                sol.unassigned.add(node_id)


    return True

def regret2_best_position_insertion(sol: Solution, removed_list, dist_fn, GOODS: List[str],
                                    vehicles_json=None, DEPOT_NODE_TO_INTERNAL=None) -> bool:
    processing_list = removed_list[:]
    while processing_list:
        candidates = []
        remainders_for_next_round = []

        for (node_id, alloc_vec) in processing_list: # <-- MARKET MOD
            if sum(alloc_vec) <= 0:
                continue
            two_best = []
            for r in sol.routes:
                if not feasible_add_chunk_to_route(r, alloc_vec):
                    continue
                for pos in range(1, len(r.path)):
                    delta_d = delta_insert_edge(r, node_id, dist_fn, pos) # <-- MARKET MOD
                    if delta_d >= HUGE:
                        continue
                    delta_cost = delta_d * r.veh_cost
                    two_best.append((delta_cost, r, delta_d, pos))
            two_best.sort(key=lambda x: x[0])

            if len(two_best) == 0:
                # Không tìm thấy vị trí chèn
                if vehicles_json and DEPOT_NODE_TO_INTERNAL:
                    done, rem_vec = _open_new_trip_and_fit(sol, node_id, alloc_vec, vehicles_json, DEPOT_NODE_TO_INTERNAL, dist_fn, GOODS) # <-- MARKET MOD
                    if not done and sum(rem_vec) > 0:
                        remainders_for_next_round.append((node_id, rem_vec)) # <-- MARKET MOD
                else:
                    remainders_for_next_round.append((node_id, alloc_vec)) # Gửi lại cho vòng sau
            else:
                candidates.append((node_id, alloc_vec, two_best[:2])) # <-- MARKET MOD

        def regret_score(lst):
            if not lst: return -HUGE
            if len(lst) == 1: return 1e9 # Ưu tiên cao nếu chỉ có 1 lựa chọn
            return lst[1][0] - lst[0][0] # Regret = cost(thứ 2) - cost(tốt nhất)

        candidates.sort(key=lambda item: regret_score(item[2]), reverse=True)
        
        # Set để theo dõi các route đã bị chèn (tránh chèn 2 node vào 1 route/vị trí?)
        # Tạm thời không cần, vì ta duyệt qua 'candidates'
        
        for (node_id, alloc_vec, best_list) in candidates: # <-- MARKET MOD
            if not best_list:
                remainders_for_next_round.append((node_id, alloc_vec)) # <-- MARKET MOD
                continue
                
            # Tìm vị trí tốt nhất (best_list[0]) MÀ VẪN CÒN FEASIBLE
            # (Có thể route đã bị đầy lên bởi 1 node khác trong vòng lặp này)
            delta_cost, r, delta_d, pos = best_list[0]
            if not feasible_add_chunk_to_route(r, alloc_vec):
                # Nếu vị trí tốt nhất không còn feasible, thử vị trí thứ 2 (nếu có)
                if len(best_list) > 1:
                    delta_cost_2, r_2, delta_d_2, pos_2 = best_list[1]
                    if feasible_add_chunk_to_route(r_2, alloc_vec):
                        # Dùng vị trí thứ 2
                        apply_insert_at(r_2, node_id, alloc_vec, delta_d_2, pos_2)
                        if node_id in sol.market_remaining:
                            sub_vector_inplace(sol.market_remaining[node_id], alloc_vec)
                        continue # Đã chèn xong node này
                
                # Cả 2 vị trí đều không feasible, đưa vào vòng sau
                remainders_for_next_round.append((node_id, alloc_vec)) # <-- MARKET MOD
                continue

            # Vị trí tốt nhất vẫn feasible
            apply_insert_at(r, node_id, alloc_vec, delta_d, pos) # <-- MARKET MOD
            # <-- MARKET MOD: Renamed
            if node_id in sol.market_remaining:
                sub_vector_inplace(sol.market_remaining[node_id], alloc_vec)

        processing_list = remainders_for_next_round
        
        # Xử lý an toàn: Nếu không có gì thay đổi, thoát vòng lặp
        if not candidates and not remainders_for_next_round:
             break # Tất cả đã xong
        if not candidates and remainders_for_next_round:
             # Còn lại nhưng không tìm thấy candidates (ví dụ: không thể mở route)
             for (node_id, alloc_vec) in remainders_for_next_round:
                 if node_id not in sol.market_remaining:
                     sol.market_remaining[node_id] = [0] * sol.n_goods
                 add_vector_inplace(sol.market_remaining[node_id], alloc_vec)
                 sol.unassigned.add(node_id)
             break # Dừng

    return True

# ======================== FEASIBILITY SWEEP ========================
def force_collect_all(sol: Solution, vehicles_json, dist_fn, DEPOT_NODE_TO_INTERNAL: Dict[int,int], GOODS: List[str]) -> None:
    # Đổi tên: Đây là "force_deliver_all"
    
    # Phải có ít nhất 1 depot_node
    if not DEPOT_NODE_TO_INTERNAL:
        # Nếu không có depot, không thể làm gì
        for mid, vec in sol.market_remaining.items():
             if sum(vec) > 0:
                 sol.unassigned.add(mid)
                 sol.market_remaining[mid] = [0]*sol.n_goods
        return

    # Lấy depot_node đầu tiên từ map
    depot_node = list(DEPOT_NODE_TO_INTERNAL.keys())[0]
    depot_internal = DEPOT_NODE_TO_INTERNAL[depot_node]

    best_v, base_cap_vec, veh_cost, veh_idx = pick_cheapest_vehicle(vehicles_json, depot_internal, GOODS)
    if best_v is None:
        # <-- MARKET MOD: Renamed
        for mid, vec in sol.market_remaining.items():
            if sum(vec) > 0:
                sol.unassigned.add(mid)
                sol.market_remaining[mid] = [0]*sol.n_goods
        return
    
    veh_type = best_v.get("vehicle_type", "type_1")

    while True:
        # <-- MARKET MOD: Renamed
        todo = [(mid, vec) for mid, vec in sol.market_remaining.items() if sum(vec) > 0]
        if not todo:
            break
        mid, rem_vec = min(todo, key=lambda t: dist_fn(depot_node, t[0])) # gần depot hơn

        if dist_fn(depot_node, mid) >= HUGE or dist_fn(mid, depot_node) >= HUGE:
            sol.unassigned.add(mid)
            sol.market_remaining[mid] = [0]*sol.n_goods # <-- MARKET MOD
            continue

        fit_vec, _ = fit_alloc_into_cap(rem_vec, base_cap_vec)
        if sum(fit_vec) <= 0:
            sol.unassigned.add(mid)
            sol.market_remaining[mid] = [0]*sol.n_goods # <-- MARKET MOD
            continue

        new_r = Route.new(depot_node, veh_idx, veh_cost, base_cap_vec, GOODS, vehicle_type=veh_type)
        delta_d = delta_insert_edge(new_r, mid, dist_fn, insert_pos=1)
        if delta_d >= HUGE:
            sol.unassigned.add(mid)
            sol.market_remaining[mid] = [0]*sol.n_goods # <-- MARKET MOD
            continue

        apply_insert_at(new_r, mid, fit_vec, delta_d, insert_pos=1)
        sol.routes.append(new_r)
        sub_vector_inplace(sol.market_remaining[mid], fit_vec) # <-- MARKET MOD

# ======================== ALNS KHỞI TẠO & VÒNG LẶP ========================
def build_initial_solution_for_cluster(
    cluster,
    vehicles_json,
    vehicle_costs,
    dist_fn,
    demand_map,
    min_cap_scaled,
    DEPOT_NODE_TO_INTERNAL: Dict[int, int],
    GOODS: List[str]
) -> Solution:
    depot_node = int(cluster["depot_node_id"])
    
    if depot_node not in DEPOT_NODE_TO_INTERNAL:
        print(f"LỖI: Depot ID {depot_node} trong cluster không có trong map DEPOT_NODE_TO_INTERNAL.")
        key_to_use = "assigned_markets" if "assigned_markets" in cluster else "assigned_vendors"
        markets = [int(v) for v in cluster.get(key_to_use, [])]
        market_remaining = {}
        for mid in markets:
            vec = [int(round(float(demand_map.get(mid, {}).get(g, 0.0))*SCALE)) for g in GOODS]
            market_remaining[mid] = vec
        return Solution(routes=[], unassigned=set(markets), market_remaining=market_remaining, n_goods=len(GOODS), cost=0.0)

    depot_internal = DEPOT_NODE_TO_INTERNAL[depot_node]
    
    # <-- MARKET MOD: Renamed variable, and use robust key finding
    key_to_use = "assigned_markets" if "assigned_markets" in cluster else "assigned_vendors"
    markets = [int(v) for v in cluster.get(key_to_use, [])]
    n_goods = len(GOODS)

    vehicles = vehicles_of_depot(vehicles_json, depot_internal)
    
    market_remaining: Dict[int, List[int]] = {} # <-- MARKET MOD
    for mid in markets: # <-- MARKET MOD
        vec = [int(round(float(demand_map.get(mid, {}).get(g, 0.0))*SCALE)) for g in GOODS]
        market_remaining[mid] = vec # <-- MARKET MOD

    if not vehicles:
        print(f"Warning: Không có xe cho depot {depot_node} (internal {depot_internal}).")
        sol = Solution(routes=[], unassigned=set(markets), market_remaining=market_remaining, n_goods=n_goods, cost=0.0) # <-- MARKET MOD
        return sol

    routes: List[Route] = []
    # Khởi tạo routes (có thể là rỗng)
    for vi, v in enumerate(vehicles):
        cap_vec = vehicle_capacity_vec(v, GOODS)
        veh_cost = vehicle_cost_per_km(v, vehicles_json["meta"]["vehicle_costs"])
        veh_type = v.get("vehicle_type", "type_1")
        # Khởi tạo route rỗng, chỉ đi và về
        routes.append(Route.new(depot_node, vi, veh_cost, cap_vec, GOODS, vehicle_type=veh_type))

    per_caps_total = [sum(vehicle_capacity_vec(v, GOODS)) for v in vehicles]
    
    # Xử lý trường hợp không có capacity
    if not per_caps_total or min(per_caps_total) <= 0:
        print(f"Warning: Xe của depot {depot_node} không có capacity.")
        min_cap = 1 # Đặt giá trị mặc định
    else:
        min_cap = int(max(1, floor(min(per_caps_total))))
        
    chunk_nodes, chunk_qty_scaled, _ = integer_split_scaled_safe(markets, demand_map, min_cap_scaled=min_cap, GOODS=GOODS) # <-- MARKET MOD

    processing_list = list(zip(chunk_nodes, chunk_qty_scaled))

    while processing_list:
        (mid, qty_scaled) = processing_list.pop(0) # <-- MARKET MOD
        
        # Kiểm tra xem market này còn hàng để phân bổ không
        if mid not in market_remaining or sum(market_remaining[mid]) <= 0:
            continue
            
        alloc = allocate_chunk_vector(market_remaining[mid], qty_scaled, GOODS) # <-- MARKET MOD
        if sum(alloc) <= 0:
            continue

        best = None
        for r in routes:
            if not feasible_add_chunk_to_route(r, alloc):
                continue
            for pos in range(1, len(r.path)):
                delta_d = delta_insert_edge(r, mid, dist_fn, pos) # <-- MARKET MOD
                if delta_d >= HUGE:
                    continue
                delta_cost = delta_d * r.veh_cost
                if (best is None) or (delta_cost < best[0]):
                    best = (delta_cost, r, delta_d, pos)

        if best is not None:
            _, r, delta_d, pos = best
            apply_insert_at(r, mid, alloc, delta_d, pos) # <-- MARKET MOD
            sub_vector_inplace(market_remaining[mid], alloc) # <-- MARKET MOD
        else:
            # Không tìm thấy route_đang_mở nào phù hợp, thử mở route mới
            best_v, base_cap_vec, veh_cost, veh_idx = pick_cheapest_vehicle(vehicles_json, depot_internal, GOODS)
            if best_v is None:
                # Không thể mở route mới, chunk này bị kẹt
                continue
            
            veh_type = best_v.get("vehicle_type", "type_1")
            new_r = Route.new(depot_node, veh_idx, veh_cost, base_cap_vec, GOODS, vehicle_type=veh_type)
            
            fit_vec, rem_vec = fit_alloc_into_cap(alloc, new_r.cap_vec)
            if sum(fit_vec) <= 0:
                # Xe rẻ nhất cũng không chở nổi chunk
                continue
                
            if dist_fn(depot_node, mid) >= HUGE or dist_fn(mid, depot_node) >= HUGE: # <-- MARKET MOD
                continue
                
            delta_d = delta_insert_edge(new_r, mid, dist_fn, insert_pos=1) # <-- MARKET MOD
            if delta_d < HUGE:
                apply_insert_at(new_r, mid, fit_vec, delta_d, insert_pos=1) # <-- MARKET MOD
                sub_vector_inplace(market_remaining[mid], fit_vec) # <-- MARKET MOD
                routes.append(new_r)
                if sum(rem_vec) > 0:
                    # Đưa phần còn lại của chunk vào đầu danh sách
                    processing_list.insert(0, (mid, int(sum(rem_vec)))) 
            else:
                # Không thể chèn vào route mới (ví dụ: bị chặn)
                pass

    sol = Solution(routes, set(), market_remaining, n_goods) # <-- MARKET MOD
    force_collect_all(sol, vehicles_json, dist_fn, DEPOT_NODE_TO_INTERNAL, GOODS)
    recompute_all_cost(sol, dist_fn)
    return sol

def roulette_select(weights: List[float]) -> int:
    s = sum(weights)
    if s <= 0:
        return int(random.random() * len(weights))
    threshold = random.random() * s
    acc = 0.0
    for i, w in enumerate(weights):
        acc += w
        if acc >= threshold:
            return i
    return len(weights)-1

def update_weights(weights: List[float], scores: List[float], counts: List[int], rho: float = RHO) -> None:
    for i in range(len(weights)):
        avg = scores[i] / max(1, counts[i])
        weights[i] = (1 - rho)*weights[i] + rho*avg
        scores[i] = 0.0
        counts[i] = 0

def alns_optimize_cluster(
    cluster,
    vehicles_json,
    dist_fn,
    demand_map,
    DEPOT_NODE_TO_INTERNAL: Dict[int, int],
    GOODS: List[str],
    max_iter=5000,
    destroy_ratio=0.15,
    cooling=0.998
):
    depot_node = int(cluster["depot_node_id"])
    print(f"\n--- Bắt đầu tối ưu cho Depot {depot_node} ---")
    
    # <-- MARKET MOD
    key_to_use = "assigned_markets" if "assigned_markets" in cluster else "assigned_vendors"
    assigned_nodes = [int(v) for v in cluster.get(key_to_use, [])]
    
    if depot_node not in DEPOT_NODE_TO_INTERNAL:
        print(f"LỖI: Depot ID {depot_node} không có trong map.")
        return {"depot_node": depot_node, "routes": [], "distance_cost_sum": 0.0, "unassigned_markets": assigned_nodes}

    depot_internal = DEPOT_NODE_TO_INTERNAL[depot_node]
    vehicles = vehicles_of_depot(vehicles_json, depot_internal)
    
    if not vehicles:
        print(f"Info: Không có xe cho depot {depot_node}.")
        return {"depot_node": depot_node, "routes": [], "distance_cost_sum": 0.0, "unassigned_markets": assigned_nodes}

    per_caps_total = [sum(vehicle_capacity_vec(v, GOODS)) for v in vehicles]
    if not per_caps_total or min(per_caps_total) <= 0:
        print(f"LỖI: Xe của depot {depot_node} không có capacity.")
        return {"depot_node": depot_node, "routes": [], "distance_cost_sum": 0.0, "unassigned_markets": assigned_nodes}

    min_cap = int(max(1, floor(min(per_caps_total))))

    curr = build_initial_solution_for_cluster(
        cluster, vehicles_json, vehicles_json["meta"]["vehicle_costs"],
        dist_fn, demand_map, min_cap,
        DEPOT_NODE_TO_INTERNAL, GOODS
    )
    best = curr.clone()
    best_cost = recompute_all_cost(best, dist_fn)
    curr_cost = best_cost

    if best_cost >= HUGE:
        print(f"Info: Không thể tạo solution ban đầu cho depot {depot_node}. Chi phí: {best_cost}")
        # <-- MARKET MOD
        return {"depot_node": depot_node, "routes": [], "distance_cost_sum": 0.0, "unassigned_markets": list(best.unassigned)}

    # <-- MARKET MOD
    print(f"Depot {depot_node}: Chi phí ban đầu = {best_cost:.2f} (Không giao được: {len(best.unassigned)})")

    destroy_ops = [
        ("random_removal", lambda s: random_removal(s, dist_fn, remove_ratio=destroy_ratio)),
        ("worst_removal",  lambda s: worst_removal(s, dist_fn, remove_ratio=destroy_ratio)),
        ("shaw_removal",   lambda s: shaw_removal(s, dist_fn, remove_ratio=destroy_ratio)),
    ]
    repair_ops = [
        ("greedy",   lambda s, rem: greedy_best_position_insertion(s, rem, dist_fn, GOODS, vehicles_json, DEPOT_NODE_TO_INTERNAL)),
        ("regret-2", lambda s, rem: regret2_best_position_insertion(s, rem, dist_fn, GOODS, vehicles_json, DEPOT_NODE_TO_INTERNAL)),
    ]
    w_d = [1.0]*len(destroy_ops)
    w_r = [1.0]*len(repair_ops)
    scr_d = [0.0]*len(destroy_ops)
    cnt_d = [0]*len(destroy_ops)
    scr_r = [0.0]*len(repair_ops)
    cnt_r = [0]*len(repair_ops)

    T = max(1.0, (best_cost % HUGE) * 0.05)
    if T == 0: T = 1.0 # Tránh T=0
        
    iter_since_last_weight_update = 0
    best_iter = 0

    for it in range(max_iter):
        di = roulette_select(w_d)
        ri = roulette_select(w_r)

        trial = curr.clone()
        removed = destroy_ops[di][1](trial)
        if not removed:
            continue

        repair_ops[ri][1](trial, removed)
        new_cost = recompute_all_cost(trial, dist_fn)

        improved_curr = new_cost < curr_cost
        improved_best = new_cost < best_cost
        reward = REWARD_GLOBAL_BEST if improved_best else (REWARD_IMPROVED if improved_curr else REWARD_ACCEPTED)
        scr_d[di] += reward; cnt_d[di] += 1
        scr_r[ri] += reward; cnt_r[ri] += 1

        delta = new_cost - curr_cost
        accept = (delta < 0) or (random.random() < exp(-delta / max(T, 1e-9)))

        if accept:
            curr = trial
            curr_cost = new_cost
            if improved_best:
                best = trial.clone()
                best_cost = new_cost
                best_iter = it

        T *= cooling
        iter_since_last_weight_update += 1
        if iter_since_last_weight_update >= SEGMENT_LENGTH:
            update_weights(w_d, scr_d, cnt_d, rho=RHO)
            update_weights(w_r, scr_r, cnt_r, rho=RHO)
            iter_since_last_weight_update = 0

        if it - best_iter >= EARLY_STOP_PATIENCE:
            print(f"Depot {depot_node}: Dừng sớm tại iter {it} (không cải thiện từ iter {best_iter})")
            break

    force_collect_all(best, vehicles_json, dist_fn, DEPOT_NODE_TO_INTERNAL, GOODS)
    best_cost = recompute_all_cost(best, dist_fn)

    routes_out, total = [], 0.0
    for r in best.routes:
        d = r.dist_cache
        if d <= 0 or d >= HUGE:
            continue
        dist_cost = d * r.veh_cost
        routes_out.append({
            "vehicle_id": f"depot{DEPOT_NODE_TO_INTERNAL[depot_node]}_veh{r.vehicle_idx}",
            "vehicle_cost_per_km": r.veh_cost,
            "vehicle_type": r.vehicle_type,
            "path_node_ids": r.path[:],
            "distance_raw": float(d),
            "route_cost": float(dist_cost)
        })
        total += dist_cost

    if best.unassigned:
        # <-- MARKET MOD: Changed "gom" (collect) to "giao" (deliver)
        print(f"⚠️ Depot {depot_node}: Không thể giao (unreachable) {len(best.unassigned)} markets: {sorted(list(best.unassigned))}")

    # <-- MARKET MOD
    print(f"Depot {depot_node}: Hoàn thành. Chi phí tốt nhất = {total:.2f} (Không giao được: {len(best.unassigned)})")
    # <-- MARKET MOD
    return {"depot_node": depot_node, "routes": routes_out, "distance_cost_sum": total, "unassigned_markets": list(best.unassigned)}

# ======================== EXPAND FULL PATH CHO OUTPUT ========================
def expand_route_path(path_ids: List[int], path_map: Dict[Tuple[int,int], List[int]]) -> List[int]:
    """
    Biến [a, v1, v2, ..., depot] thành chuỗi node đầy đủ bằng cách thay mỗi cạnh (u->v)
    bằng path_map[u,v] nếu có (bỏ lặp u), ngược lại giữ (u,v) như cũ.
    """
    if not path_ids or len(path_ids) == 1:
        return path_ids[:]
    out = [path_ids[0]]
    for i in range(len(path_ids)-1):
        u, v = path_ids[i], path_ids[i+1]
        key = (u, v)
        if key in path_map and len(path_map[key]) >= 2:
            seg = path_map[key]
            # thêm toàn bộ trừ node đầu (tránh lặp)
            out.extend(seg[1:])
        else:
            # không có path_map -> giữ cạnh trực tiếp
            out.append(v)
    return out

def main():
    parser = argparse.ArgumentParser(description="ALNS Delivery + On-demand shortest-path for blocked depot<->market edges (expand path at output)") # <-- MARKET MOD
    parser.add_argument("--base_dir", type=str, default=r"C:\Users\thaip\Desktop\VRP4\data\JSON\osaba-50-instances")
    
    # --- FIXED LINE --- (Sửa lỗi 'AttributeError: add_default')
    parser.add_argument("--cluster_file", type=str, default="market_cluster_50.json") # <-- User's file
    # --- END FIX ---
    
    parser.add_argument("--nodes_file", type=str, default="osaba_50_standard.json")
    parser.add_argument("--matrix_file", type=str, default="osaba_50_graph_matrix.json")
    parser.add_argument("--vehicles_file", type=str, default="osaba_50_vehicles.json")
    parser.add_argument("--max_iter", type=int, default=2000)
    parser.add_argument("--destroy_ratio", type=float, default=0.18)
    parser.add_argument("--cooling", type=float, default=0.998)
    args = parser.parse_args() if 'get_ipython' not in globals() else argparse.Namespace(
        base_dir=r"C:\Users\thaip\Desktop\VRP4\data\JSON\osaba-50-instances",
        cluster_file="market_cluster_50.json", # <-- User's file
        nodes_file="osaba_50_standard.json",
        matrix_file="osaba_50_graph_matrix.json",
        vehicles_file="osaba_50_vehicles.json",
        max_iter=8000,
        destroy_ratio=0.25,
        cooling=0.997
    )

    random.seed(SEED)

    base_dir = Path(args.base_dir)
    path_cluster = base_dir / args.cluster_file
    path_nodes = base_dir / args.nodes_file
    path_matrix = base_dir / args.matrix_file
    path_vehicles = base_dir / args.vehicles_file
    
    # Kiểm tra file
    if not path_cluster.exists():
        print(f"LỖI: Không tìm thấy file cluster: {path_cluster}")
        return
    if not path_nodes.exists():
        print(f"LỖI: Không tìm thấy file nodes: {path_nodes}")
        return
    if not path_matrix.exists():
        print(f"LỖI: Không tìm thấy file matrix: {path_matrix}")
        return
    if not path_vehicles.exists():
        print(f"LỖI: Không tìm thấy file vehicles: {path_vehicles}")
        return

    print("Đang tải file...")
    nodes = load_json(path_nodes)
    clusters = load_json(path_cluster)
    vehicles = load_json(path_vehicles)
    dist_json = load_json(path_matrix)

    GOODS = nodes["meta"]["commodities"]
    depot_nodes = [int(n["id"]) for n in nodes["nodes"] if n["type"] == "depot"]
    depot_nodes.sort()
    DEPOT_NODE_TO_INTERNAL = {node_id: i for i, node_id in enumerate(depot_nodes)}

    floyd_path = path_matrix.with_name(path_matrix.stem + "_floyd.json")
    dist_num, path_map = preprocess_only_for_blocked_pairs(nodes, clusters, dist_json, floyd_path)
    id2idx = build_id_to_idx(nodes)
    dist_fn = dist_accessor(dist_num, id2idx, path_map)
    demand = get_market_demand(nodes, GOODS) # <-- MARKET MOD: Call new function

    total_cost = 0.0
    results, total_unassigned = [], []

    # --- START ROBUST FIX ---
    # (Đoạn này đã được sửa để xử lý file cluster không có key "clusters")
    cluster_list = clusters.get("clusters")
    if cluster_list is None:
        if isinstance(clusters, list):
            cluster_list = clusters
        else:
            # Nếu vẫn không tìm thấy, thử tìm key đầu tiên là list
            found_list = [v for v in clusters.values() if isinstance(v, list)]
            if found_list:
                cluster_list = found_list[0]
            else:
                cluster_list = []
                print(f"CẢNH BÁO: Không tìm thấy key 'clusters' hoặc một list cluster nào trong file {path_cluster.name}")
    # --- END ROBUST FIX ---

    for c in cluster_list:
        res = alns_optimize_cluster(
            c, vehicles_json=vehicles, dist_fn=dist_fn,
            demand_map=demand, DEPOT_NODE_TO_INTERNAL=DEPOT_NODE_TO_INTERNAL,
            GOODS=GOODS, max_iter=args.max_iter, destroy_ratio=args.destroy_ratio,
            cooling=args.cooling
        )
        results.append(res)
        total_cost += float(res.get("distance_cost_sum", 0.0))
        if res.get("unassigned_markets"): # <-- MARKET MOD
            total_unassigned.extend(res["unassigned_markets"]) # <-- MARKET MOD

    # ===== Expand & GOM THEO vehicle_type =====
    print("\n============ SUMMARY VEHICLE TYPE ROUTES ============")
    for cluster_res in results:
        depot = cluster_res["depot_node"]
        routes = cluster_res["routes"]
        print(f"\nDepot {depot}:")
        type_summary = {}

        for r in routes:
            # Expand đường nếu có path_map
            r["path_node_ids"] = expand_route_path(r["path_node_ids"], path_map)
            
            # Lấy loại xe trực tiếp từ kết quả (đã được thêm vào ở alns_optimize_cluster)
            veh_type = r.get("vehicle_type", "unknown")
            
            # Gom nhóm
            if veh_type not in type_summary:
                type_summary[veh_type] = []
            type_summary[veh_type].append(r["path_node_ids"])

        # In console
        for vt, trips in type_summary.items():
            print(f"  {vt}:")
            for i, trip in enumerate(trips, 1):
                path_str = " → ".join(map(str, trip))
                print(f"    Trip {i} → {path_str}")

        cluster_res["vehicle_type_summary"] = type_summary  # thêm vào output JSON

    # ===== Lưu OUTPUT =====
    # <-- MARKET MOD: Use a dynamic name based on the cluster file stem
    output_filename = f"alns_routes_delivery_{path_cluster.stem}.json"
    output_path = path_cluster.parent / output_filename
    
    out = {
        # <-- MARKET MOD
        "objective": "sum(distance * vehicle_type_cost) over routes (Delivery Phase)",
        "total_cost": total_cost,
        "distance_unit": "raw_from_matrix",
        "seed": SEED,
        "clusters": results,
        "total_unassigned_markets": sorted(list(set(total_unassigned))), # <-- MARKET MOD
        "preprocessed_graph": str(floyd_path.name)
    }
    save_json(output_path, out)

    print("\n" + "="*50)
    # <-- MARKET MOD
    print(f"✅ Đã lưu kết quả ALNS delivery vào: {output_path}")
    print(f"Tổng chi phí (Tất cả depot): {total_cost:.2f}")
    if total_unassigned:
        # <-- MARKET MOD
        print(f"⚠️ Không giao được {len(set(total_unassigned))} markets: {sorted(list(set(total_unassigned)))}")

if __name__ == "__main__":
    main()

Đang tải file...

--- Bắt đầu tối ưu cho Depot 6 ---
Depot 6: Chi phí ban đầu = 24788.60 (Không giao được: 0)
Depot 6: Dừng sớm tại iter 2009 (không cải thiện từ iter 9)
Depot 6: Hoàn thành. Chi phí tốt nhất = 21500.60 (Không giao được: 0)

--- Bắt đầu tối ưu cho Depot 16 ---
Depot 16: Chi phí ban đầu = 14863.00 (Không giao được: 0)
Depot 16: Dừng sớm tại iter 2000 (không cải thiện từ iter 0)
Depot 16: Hoàn thành. Chi phí tốt nhất = 14863.00 (Không giao được: 0)

--- Bắt đầu tối ưu cho Depot 25 ---
Depot 25: Chi phí ban đầu = 29965.60 (Không giao được: 0)
Depot 25: Dừng sớm tại iter 2091 (không cải thiện từ iter 91)
Depot 25: Hoàn thành. Chi phí tốt nhất = 22302.20 (Không giao được: 0)

--- Bắt đầu tối ưu cho Depot 36 ---
Depot 36: Chi phí ban đầu = 38754.20 (Không giao được: 0)
Depot 36: Dừng sớm tại iter 2477 (không cải thiện từ iter 477)
Depot 36: Hoàn thành. Chi phí tốt nhất = 36151.20 (Không giao được: 0)

--- Bắt đầu tối ưu cho Depot 45 ---
Depot 45: Chi phí ban đầu = 19096.80 (K